In [ ]:
import numpy as np
import data_io
from data_preprocessing import *
from implementations import *
import validation
import attribute_selection
import evaluators

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_FILE_PREFIX = '/content/drive/My Drive/mlproject1_higgs_data/'

In [ ]:
y, x, ids, cols = data_io.load_csv_data(f'{DATA_FILE_PREFIX}train.csv')

In [ ]:
# Make it easier to access columns by their name
col_to_index_mapping = {col_name: index - 2 for index, col_name in enumerate(cols) if index >= 2}

In [ ]:
def train_model(y, x):
  #w, _ = least_squares_GD(y, x, np.zeros((x.shape[1],)), 100, gamma=0.1)
  w, loss = least_squares(y, x)
  def predict(features):
    result = features @ w
    result[np.where(result < 0)] = -1
    result[np.where(result >= 0)] = 1
    return result
  return predict

In [ ]:
attribute_selection.backward_attribute_selector(y, x, evaluators.cross_validation_mean_acc_evaluator(train_model), 20, verbose=True)

In [ ]:
a = np.array(sorted([2, 1, 13, 8, 19, 21, 0, 5, 26, 29, 7, 22, 25, 9, 10, 3, 16, 14, 15, 23]))
b = np.array([0, 1, 2, 3, 4, 5, 6, 7, 10, 12, 13, 16, 18, 21, 22, 23, 24, 26, 28, 29])
print(a)
print(b)

In [ ]:
attribute_selection.forward_attribute_selector(y, x, evaluators.cross_validation_mean_acc_evaluator(train_model), 50, verbose=True)

In [ ]:
def train_model(y, x):
  def predict(features):
    return np.array([1] * features.shape[0])
  return predict
_ = validation.cross_validation(y, x, train_model, 10, verbose=True)

In [ ]:
tx = np.copy(x) # Recommended to copy x so it doesn't change
ty = np.copy(y)

missing_values = tx == -999.

tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

#too_many_missing_columns = np.where(np.sum(missing_values, axis=0) > 250000 * .)
#print(too_many_missing_columns)



#tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))

#tx = np.delete(tx, too_many_missing_columns, axis=1)

tx = prepend_bias_column(tx)
_ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)

In [ ]:
not_pri_keys = list(map(lambda key: col_to_index_mapping[key], filter(lambda key: 'PRI' in key, col_to_index_mapping)))
not_pri_keys

In [ ]:
removable = [['DER_mass_MMC', 0.74616], ['DER_mass_transverse_met_lep', 0.74616], ['DER_mass_vis', 0.727088], ['DER_pt_h', 0.727088], ['DER_deltaeta_jet_jet', 0.7380439999999998], ['DER_mass_jet_jet', 0.7470760000000001], ['DER_prodeta_jet_jet', 0.7468960000000001], ['DER_deltar_tau_lep', 0.7461], ['DER_pt_tot', 0.746904], ['DER_sum_pt', 0.73704], ['DER_pt_ratio_lep_tau', 0.7468960000000001], ['DER_met_phi_centrality', 0.7468440000000001], ['DER_lep_eta_centrality', 0.7442599999999999], ['PRI_tau_pt', 0.750008], ['PRI_tau_eta', 0.744848], ['PRI_tau_phi', 0.7468440000000001], ['PRI_lep_pt', 0.746916], ['PRI_lep_eta', 0.746912], ['PRI_lep_phi', 0.7468440000000001], ['PRI_met', 0.746864], ['PRI_met_phi', 0.7467560000000001], ['PRI_met_sumet', 0.745808], ['PRI_jet_leading_pt', 0.746824], ['PRI_jet_leading_eta', 0.7465839999999999], ['PRI_jet_leading_phi', 0.746904], ['PRI_jet_subleading_pt', 0.746872], ['PRI_jet_subleading_eta', 0.7468680000000001], ['PRI_jet_subleading_phi', 0.746944], ['PRI_jet_all_pt', 0.746836]]
removable = sorted(filter(lambda x: x[1] > 0.75, removable), key=lambda x: -x[1])
to_be_removed = list(map(lambda x: col_to_index_mapping[x[0]], removable))
to_be_removed

In [ ]:
col_to_index_mapping

In [ ]:
#pred = map_values(np.random.randint(0, 2, y.size), {0: -1, 1: 1})
pred = np.array([-1] * y.size)

In [ ]:
import metrics
# Tests metrics (fbeta_score, precision and recall might return "RuntimeWarning: invalid value encountered in long_scalars"
#     which functions take care of (these are nan cases) by returning 0)
print('Acc:', metrics.accuracy(y, pred))
print('Fbeta:', metrics.fbeta_score(y, pred))
print('Precision:', metrics.precision(y, pred))
print('Recall:', metrics.recall(y, pred))

In [ ]:
def train_model(y, x):
  #w, _ = least_squares_GD(y, x, np.zeros((x.shape[1],)), 100, gamma=0.1)
  w, loss = least_squares(y, x)
  def predict(features):
    result = features @ w
    result[np.where(result < 0)] = -1
    result[np.where(result >= 0)] = 1
    return result
  return predict

In [ ]:
best_degrees = {}
for key in col_to_index_mapping:
  if 'PRI_jet_num' not in key:
    best_degree_val = 0
    best_degree = 1
    for degree in range(1, 13):
      col_index = col_to_index_mapping[key]
      if col_index > col_to_index_mapping['PRI_jet_num']:
        col_index -= 1
      
      tx = np.copy(x) # Recommended to copy x so it doesn't change
      ty = np.copy(y)

      missing_values = tx == -999.

      tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
      tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
      tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

      tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      tx = prepend_bias_column(tx)
      print(f'Key \'{key}\', degree {degree}:')
      accs, _ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)
      mean_acc = np.mean(accs)
      if mean_acc > best_degree_val:
        best_degree_val = mean_acc
        best_degree = degree
    best_degrees[key] = (best_degree, best_degree_val)
print(best_degrees)


In [ ]:
best_single_degrees = []

tx = np.copy(x) # Recommended to copy x so it doesn't change
ty = np.copy(y)

missing_values = tx == -999.

tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

for key in col_to_index_mapping:
  if 'PRI_jet_num' not in key:
    for degree in range(2, 21):
      col_index = col_to_index_mapping[key]
      if col_index > col_to_index_mapping['PRI_jet_num']:
        col_index -= 1
      
      ttx = np.copy(tx)

      tx_col = ttx[:, [col_index]] ** degree
      ttx = np.append(ttx, tx_col, axis=1)
      #tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      ttx = prepend_bias_column(ttx)
      print(f'Key \'{key}\', degree {degree}:')
      try:
        accs, _ = validation.cross_validation(ty, ttx, train_model, 10, verbose=True)
        mean_acc = np.mean(accs)
        best_single_degrees.append([key, degree, mean_acc])
      except:
        print('Failed')
print(best_single_degrees)


In [ ]:
exponentials = []

tx = np.copy(x) # Recommended to copy x so it doesn't change
ty = np.copy(y)

missing_values = tx == -999.

tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

for key in col_to_index_mapping:
  if 'PRI_jet_num' not in key:
    #for degree in range(2, 21):
      col_index = col_to_index_mapping[key]
      if col_index > col_to_index_mapping['PRI_jet_num']:
        col_index -= 1
      
      ttx = np.copy(tx)

      tx_col = np.tanh(ttx[:, [col_index]])
      ttx = np.append(ttx, tx_col, axis=1)
      #tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      ttx = prepend_bias_column(ttx)
      print(f'Key \'{key}\':')
      try:
        accs, _ = validation.cross_validation(ty, ttx, train_model, 10, verbose=True)
        mean_acc = np.mean(accs)
        exponentials.append([key, mean_acc])
      except:
        print('Failed')
print(exponentials)


In [ ]:
exponentials = []

tx = np.copy(x) # Recommended to copy x so it doesn't change
ty = np.copy(y)

missing_values = tx == -999.

tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

for key in col_to_index_mapping:
  if 'PRI_jet_num' not in key:
    #for degree in range(2, 21):
      col_index = col_to_index_mapping[key]
      if col_index > col_to_index_mapping['PRI_jet_num']:
        col_index -= 1
      
      ttx = np.copy(tx)

      #tx_col = np.tanh(ttx[:, [col_index]])
      #ttx = np.append(ttx, tx_col, axis=1)
      ttx = np.delete(ttx, col_to_index_mapping_copy[key], axis=1)
      #tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      ttx = prepend_bias_column(ttx)
      print(f'Key \'{key}\':')
      try:
        accs, _ = validation.cross_validation(ty, ttx, train_model, 10, verbose=True)
        mean_acc = np.mean(accs)
        exponentials.append([key, mean_acc])
      except:
        print('Failed')
print(exponentials)

In [ ]:
pair = {}

tx = np.copy(x) # Recommended to copy x so it doesn't change
ty = np.copy(y)

missing_values = tx == -999.

tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

for key1 in col_to_index_mapping:
  for key2 in col_to_index_mapping:
    if 'PRI_jet_num' not in [key1, key2]: #and f'{key1},{key2}' not in pair:
      col1_index = col_to_index_mapping[key1]
      if col1_index > col_to_index_mapping['PRI_jet_num']:
        col1_index -= 1
      col2_index = col_to_index_mapping[key2]
      if col2_index > col_to_index_mapping['PRI_jet_num']:
        col2_index -= 1
      
      ttx = np.copy(tx)

      tx_col1 = np.abs(ttx[:, [col1_index]])
      tx_col2 = np.abs(ttx[:, [col2_index]])
      ttx = np.append(ttx, tx_col1 - tx_col2, axis=1)
      #tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      ttx = prepend_bias_column(ttx)
      print(f'Keys \'{key1}\' and \'{key2}\':')
      try:
        accs, _ = validation.cross_validation(ty, ttx, train_model, 10, verbose=True)
        mean_acc = np.mean(accs)
        pair[f'{key1},{key2}'] = mean_acc
      except:
        print('Failed')
print(pair)


In [ ]:
pairs_arr = sorted([(key, val) for key, val in pair.items()], key=lambda x: -x[1])
pairs_arr

In [ ]:
best_single_degrees = [['DER_mass_MMC', 2, 0.750048], ['DER_mass_MMC', 3, 0.748548], ['DER_mass_MMC', 4, 0.747448], ['DER_mass_MMC', 5, 0.747096], ['DER_mass_MMC', 6, 0.7469560000000001], ['DER_mass_MMC', 7, 0.746916], ['DER_mass_MMC', 8, 0.746888], ['DER_mass_MMC', 9, 0.7468760000000001], ['DER_mass_MMC', 10, 0.7468760000000001], ['DER_mass_MMC', 11, 0.746872], ['DER_mass_MMC', 12, 0.746868], ['DER_mass_MMC', 13, 0.746856], ['DER_mass_MMC', 14, 0.746856], ['DER_mass_MMC', 15, 0.746864], ['DER_mass_MMC', 16, 0.74686], ['DER_mass_MMC', 17, 0.746864], ['DER_mass_MMC', 18, 0.746864], ['DER_mass_MMC', 19, 0.746868], ['DER_mass_MMC', 20, 0.746868], ['DER_mass_transverse_met_lep', 2, 0.747008], ['DER_mass_transverse_met_lep', 3, 0.7476], ['DER_mass_transverse_met_lep', 4, 0.747188], ['DER_mass_transverse_met_lep', 5, 0.7471440000000001], ['DER_mass_transverse_met_lep', 6, 0.747068], ['DER_mass_transverse_met_lep', 7, 0.746972], ['DER_mass_transverse_met_lep', 8, 0.7469520000000001], ['DER_mass_transverse_met_lep', 9, 0.746932], ['DER_mass_transverse_met_lep', 10, 0.746916], ['DER_mass_transverse_met_lep', 11, 0.7468960000000001], ['DER_mass_transverse_met_lep', 12, 0.7468920000000001], ['DER_mass_transverse_met_lep', 13, 0.74692], ['DER_mass_transverse_met_lep', 14, 0.7469119999999999], ['DER_mass_transverse_met_lep', 15, 0.746936], ['DER_mass_transverse_met_lep', 16, 0.7469239999999999], ['DER_mass_transverse_met_lep', 17, 0.746916], ['DER_mass_transverse_met_lep', 18, 0.746908], ['DER_mass_transverse_met_lep', 19, 0.746908], ['DER_mass_transverse_met_lep', 20, 0.7469159999999999], ['DER_mass_vis', 2, 0.748024], ['DER_mass_vis', 3, 0.746924], ['DER_mass_vis', 4, 0.7468959999999999], ['DER_mass_vis', 5, 0.7469080000000001], ['DER_mass_vis', 6, 0.7468960000000001], ['DER_mass_vis', 7, 0.7468600000000001], ['DER_mass_vis', 8, 0.7468680000000001], ['DER_mass_vis', 9, 0.7468720000000001], ['DER_mass_vis', 10, 0.746856], ['DER_mass_vis', 11, 0.746848], ['DER_mass_vis', 12, 0.746848], ['DER_mass_vis', 13, 0.7468440000000001], ['DER_mass_vis', 14, 0.7468480000000001], ['DER_mass_vis', 15, 0.7468480000000001], ['DER_mass_vis', 16, 0.7468440000000001], ['DER_mass_vis', 17, 0.7468480000000001], ['DER_mass_vis', 18, 0.746852], ['DER_mass_vis', 19, 0.7468480000000002], ['DER_mass_vis', 20, 0.746848], ['DER_pt_h', 2, 0.747944], ['DER_pt_h', 3, 0.7472000000000001], ['DER_pt_h', 4, 0.747032], ['DER_pt_h', 5, 0.746916], ['DER_pt_h', 6, 0.74692], ['DER_pt_h', 7, 0.74694], ['DER_pt_h', 8, 0.74694], ['DER_pt_h', 9, 0.7469319999999999], ['DER_pt_h', 10, 0.7469319999999999], ['DER_pt_h', 11, 0.7469319999999999], ['DER_pt_h', 12, 0.7469319999999999], ['DER_pt_h', 13, 0.7469319999999999], ['DER_pt_h', 14, 0.7469319999999999], ['DER_pt_h', 15, 0.7469319999999999], ['DER_pt_h', 16, 0.746936], ['DER_pt_h', 17, 0.7469319999999999], ['DER_pt_h', 18, 0.7469279999999999], ['DER_pt_h', 19, 0.7469319999999999], ['DER_pt_h', 20, 0.7469319999999999], ['DER_deltaeta_jet_jet', 2, 0.7482439999999999], ['DER_deltaeta_jet_jet', 3, 0.748236], ['DER_deltaeta_jet_jet', 4, 0.748212], ['DER_deltaeta_jet_jet', 5, 0.748208], ['DER_deltaeta_jet_jet', 6, 0.748192], ['DER_deltaeta_jet_jet', 7, 0.7481679999999999], ['DER_deltaeta_jet_jet', 8, 0.7481519999999999], ['DER_deltaeta_jet_jet', 9, 0.74816], ['DER_deltaeta_jet_jet', 10, 0.748156], ['DER_deltaeta_jet_jet', 11, 0.7481440000000001], ['DER_deltaeta_jet_jet', 12, 0.748132], ['DER_deltaeta_jet_jet', 13, 0.7481280000000001], ['DER_deltaeta_jet_jet', 14, 0.748124], ['DER_deltaeta_jet_jet', 15, 0.74814], ['DER_deltaeta_jet_jet', 16, 0.7481479999999999], ['DER_deltaeta_jet_jet', 17, 0.74812], ['DER_deltaeta_jet_jet', 18, 0.7481120000000001], ['DER_deltaeta_jet_jet', 19, 0.7481120000000001], ['DER_deltaeta_jet_jet', 20, 0.748116], ['DER_mass_jet_jet', 2, 0.746804], ['DER_mass_jet_jet', 3, 0.746828], ['DER_mass_jet_jet', 4, 0.7468279999999999], ['DER_mass_jet_jet', 5, 0.746888], ['DER_mass_jet_jet', 6, 0.7468760000000001], ['DER_mass_jet_jet', 7, 0.746884], ['DER_mass_jet_jet', 8, 0.7468239999999999], ['DER_mass_jet_jet', 9, 0.746836], ['DER_mass_jet_jet', 10, 0.746848], ['DER_mass_jet_jet', 11, 0.7468440000000001], ['DER_mass_jet_jet', 12, 0.7468600000000001], ['DER_mass_jet_jet', 13, 0.746868], ['DER_mass_jet_jet', 14, 0.746884], ['DER_mass_jet_jet', 15, 0.746884], ['DER_mass_jet_jet', 16, 0.74688], ['DER_mass_jet_jet', 17, 0.746872], ['DER_mass_jet_jet', 18, 0.746876], ['DER_mass_jet_jet', 19, 0.7468680000000001], ['DER_mass_jet_jet', 20, 0.746872], ['DER_prodeta_jet_jet', 2, 0.7473559999999999], ['DER_prodeta_jet_jet', 3, 0.747352], ['DER_prodeta_jet_jet', 4, 0.7473479999999999], ['DER_prodeta_jet_jet', 5, 0.747344], ['DER_prodeta_jet_jet', 6, 0.747324], ['DER_prodeta_jet_jet', 7, 0.7473279999999999], ['DER_prodeta_jet_jet', 8, 0.747308], ['DER_prodeta_jet_jet', 9, 0.747308], ['DER_prodeta_jet_jet', 10, 0.7473240000000001], ['DER_prodeta_jet_jet', 11, 0.747336], ['DER_prodeta_jet_jet', 12, 0.747332], ['DER_prodeta_jet_jet', 13, 0.747312], ['DER_prodeta_jet_jet', 14, 0.7473120000000001], ['DER_prodeta_jet_jet', 15, 0.747316], ['DER_prodeta_jet_jet', 16, 0.74732], ['DER_prodeta_jet_jet', 17, 0.747328], ['DER_prodeta_jet_jet', 18, 0.747352], ['DER_prodeta_jet_jet', 19, 0.7473799999999999], ['DER_prodeta_jet_jet', 20, 0.747388], ['DER_deltar_tau_lep', 2, 0.75266], ['DER_deltar_tau_lep', 3, 0.7475119999999998], ['DER_deltar_tau_lep', 4, 0.7498920000000001], ['DER_deltar_tau_lep', 5, 0.747032], ['DER_deltar_tau_lep', 6, 0.747388], ['DER_deltar_tau_lep', 7, 0.746964], ['DER_deltar_tau_lep', 8, 0.746868], ['DER_deltar_tau_lep', 9, 0.746888], ['DER_deltar_tau_lep', 10, 0.7468800000000001], ['DER_deltar_tau_lep', 11, 0.74688], ['DER_deltar_tau_lep', 12, 0.746876], ['DER_deltar_tau_lep', 13, 0.746872], ['DER_deltar_tau_lep', 14, 0.7468960000000001], ['DER_deltar_tau_lep', 15, 0.746892], ['DER_deltar_tau_lep', 16, 0.746868], ['DER_deltar_tau_lep', 17, 0.746872], ['DER_deltar_tau_lep', 18, 0.746876], ['DER_deltar_tau_lep', 19, 0.74688], ['DER_deltar_tau_lep', 20, 0.746876], ['DER_pt_tot', 2, 0.7468999999999999], ['DER_pt_tot', 3, 0.7469560000000001], ['DER_pt_tot', 4, 0.74694], ['DER_pt_tot', 5, 0.7469520000000001], ['DER_pt_tot', 6, 0.7469520000000001], ['DER_pt_tot', 7, 0.746948], ['DER_pt_tot', 8, 0.746948], ['DER_pt_tot', 9, 0.746944], ['DER_pt_tot', 10, 0.74694], ['DER_pt_tot', 11, 0.746944], ['DER_pt_tot', 12, 0.74694], ['DER_pt_tot', 13, 0.746948], ['DER_pt_tot', 14, 0.746944], ['DER_pt_tot', 15, 0.746944], ['DER_pt_tot', 16, 0.746944], ['DER_pt_tot', 17, 0.746944], ['DER_pt_tot', 18, 0.746944], ['DER_pt_tot', 19, 0.746956], ['DER_pt_tot', 20, 0.746944], ['DER_sum_pt', 2, 0.750776], ['DER_sum_pt', 3, 0.747788], ['DER_sum_pt', 4, 0.7472199999999999], ['DER_sum_pt', 5, 0.746872], ['DER_sum_pt', 6, 0.746916], ['DER_sum_pt', 7, 0.746904], ['DER_sum_pt', 8, 0.7468880000000001], ['DER_sum_pt', 9, 0.7468880000000001], ['DER_sum_pt', 10, 0.7468760000000001], ['DER_sum_pt', 11, 0.746872], ['DER_sum_pt', 12, 0.746872], ['DER_sum_pt', 13, 0.746872], ['DER_sum_pt', 14, 0.7468720000000001], ['DER_sum_pt', 15, 0.7468720000000001], ['DER_sum_pt', 16, 0.7468640000000001], ['DER_sum_pt', 17, 0.7468640000000001], ['DER_sum_pt', 18, 0.7468680000000001], ['DER_sum_pt', 19, 0.7468680000000001], ['DER_sum_pt', 20, 0.7468680000000001], ['DER_pt_ratio_lep_tau', 2, 0.748336], ['DER_pt_ratio_lep_tau', 3, 0.7468959999999999], ['DER_pt_ratio_lep_tau', 4, 0.7470079999999999], ['DER_pt_ratio_lep_tau', 5, 0.746904], ['DER_pt_ratio_lep_tau', 6, 0.74688], ['DER_pt_ratio_lep_tau', 7, 0.7468839999999999], ['DER_pt_ratio_lep_tau', 8, 0.746888], ['DER_pt_ratio_lep_tau', 9, 0.746884], ['DER_pt_ratio_lep_tau', 10, 0.746904], ['DER_pt_ratio_lep_tau', 11, 0.746856], ['DER_pt_ratio_lep_tau', 12, 0.746876], ['DER_pt_ratio_lep_tau', 13, 0.746888], ['DER_pt_ratio_lep_tau', 14, 0.7468799999999999], ['DER_pt_ratio_lep_tau', 15, 0.7468839999999999], ['DER_pt_ratio_lep_tau', 16, 0.7468839999999999], ['DER_pt_ratio_lep_tau', 17, 0.7468839999999999], ['DER_pt_ratio_lep_tau', 18, 0.7468839999999999], ['DER_pt_ratio_lep_tau', 19, 0.746888], ['DER_pt_ratio_lep_tau', 20, 0.746888], ['DER_met_phi_centrality', 2, 0.7470559999999999], ['DER_met_phi_centrality', 3, 0.74716], ['DER_met_phi_centrality', 4, 0.7472199999999999], ['DER_met_phi_centrality', 5, 0.746992], ['DER_met_phi_centrality', 6, 0.7471400000000001], ['DER_met_phi_centrality', 7, 0.7469720000000001], ['DER_met_phi_centrality', 8, 0.74712], ['DER_met_phi_centrality', 9, 0.7469720000000001], ['DER_met_phi_centrality', 10, 0.7470239999999999], ['DER_met_phi_centrality', 11, 0.7468839999999999], ['DER_met_phi_centrality', 12, 0.7469919999999999], ['DER_met_phi_centrality', 13, 0.7468839999999999], ['DER_met_phi_centrality', 14, 0.7469080000000001], ['DER_met_phi_centrality', 15, 0.746856], ['DER_met_phi_centrality', 16, 0.746852], ['DER_met_phi_centrality', 17, 0.746904], ['DER_met_phi_centrality', 18, 0.7469159999999999], ['DER_met_phi_centrality', 19, 0.7468159999999999], ['DER_met_phi_centrality', 20, 0.746836], ['DER_lep_eta_centrality', 2, 0.7457320000000001], ['DER_lep_eta_centrality', 3, 0.7469680000000001], ['DER_lep_eta_centrality', 4, 0.74702], ['DER_lep_eta_centrality', 5, 0.742964], ['DER_lep_eta_centrality', 6, 0.7468199999999999], ['DER_lep_eta_centrality', 7, 0.74684], ['DER_lep_eta_centrality', 8, 0.7468920000000001], ['DER_lep_eta_centrality', 9, 0.746852], ['DER_lep_eta_centrality', 10, 0.746732], ['DER_lep_eta_centrality', 11, 0.746848], ['DER_lep_eta_centrality', 12, 0.746804], ['DER_lep_eta_centrality', 13, 0.746872], ['DER_lep_eta_centrality', 14, 0.7468359999999999], ['DER_lep_eta_centrality', 15, 0.746864], ['DER_lep_eta_centrality', 16, 0.746828], ['DER_lep_eta_centrality', 17, 0.7468359999999999], ['DER_lep_eta_centrality', 18, 0.7468679999999999], ['DER_lep_eta_centrality', 19, 0.746828], ['DER_lep_eta_centrality', 20, 0.7468319999999999], ['PRI_tau_pt', 2, 0.7524959999999998], ['PRI_tau_pt', 3, 0.7478640000000001], ['PRI_tau_pt', 4, 0.74712], ['PRI_tau_pt', 5, 0.74696], ['PRI_tau_pt', 6, 0.74692], ['PRI_tau_pt', 7, 0.7468760000000001], ['PRI_tau_pt', 8, 0.7468800000000001], ['PRI_tau_pt', 9, 0.7468760000000001], ['PRI_tau_pt', 10, 0.746856], ['PRI_tau_pt', 11, 0.7468520000000001], ['PRI_tau_pt', 12, 0.7468480000000001], ['PRI_tau_pt', 13, 0.7468480000000001], ['PRI_tau_pt', 14, 0.7468440000000001], ['PRI_tau_pt', 15, 0.74684], ['PRI_tau_pt', 16, 0.7468359999999999], ['PRI_tau_pt', 17, 0.7468319999999999], ['PRI_tau_pt', 18, 0.7468319999999999], ['PRI_tau_pt', 19, 0.7468319999999999], ['PRI_tau_pt', 20, 0.7468319999999999], ['PRI_tau_eta', 2, 0.749588], ['PRI_tau_eta', 3, 0.746932], ['PRI_tau_eta', 4, 0.74902], ['PRI_tau_eta', 5, 0.7469399999999999], ['PRI_tau_eta', 6, 0.748464], ['PRI_tau_eta', 7, 0.746928], ['PRI_tau_eta', 8, 0.7483279999999999], ['PRI_tau_eta', 9, 0.746936], ['PRI_tau_eta', 10, 0.7478400000000001], ['PRI_tau_eta', 11, 0.7469279999999999], ['PRI_tau_eta', 12, 0.7477320000000001], ['PRI_tau_eta', 13, 0.7469239999999999], ['PRI_tau_eta', 14, 0.747504], ['PRI_tau_eta', 15, 0.746928], ['PRI_tau_eta', 16, 0.7474999999999999], ['PRI_tau_eta', 17, 0.7469199999999999], ['PRI_tau_eta', 18, 0.74742], ['PRI_tau_eta', 19, 0.74692], ['PRI_tau_eta', 20, 0.7473799999999999], ['PRI_tau_phi', 2, 0.746972], ['PRI_tau_phi', 3, 0.7470359999999999], ['PRI_tau_phi', 4, 0.746972], ['PRI_tau_phi', 5, 0.7469919999999999], ['PRI_tau_phi', 6, 0.7469319999999999], ['PRI_tau_phi', 7, 0.7469720000000001], ['PRI_tau_phi', 8, 0.746916], ['PRI_tau_phi', 9, 0.7469279999999999], ['PRI_tau_phi', 10, 0.7469439999999998], ['PRI_tau_phi', 11, 0.746888], ['PRI_tau_phi', 12, 0.746936], ['PRI_tau_phi', 13, 0.746892], ['PRI_tau_phi', 14, 0.7469319999999999], ['PRI_tau_phi', 15, 0.7468920000000001], ['PRI_tau_phi', 16, 0.746912], ['PRI_tau_phi', 17, 0.746904], ['PRI_tau_phi', 18, 0.746904], ['PRI_tau_phi', 19, 0.746892], ['PRI_tau_phi', 20, 0.7469119999999999], ['PRI_lep_pt', 2, 0.7471840000000001], ['PRI_lep_pt', 3, 0.7468760000000001], ['PRI_lep_pt', 4, 0.74684], ['PRI_lep_pt', 5, 0.746816], ['PRI_lep_pt', 6, 0.7468], ['PRI_lep_pt', 7, 0.7468079999999999], ['PRI_lep_pt', 8, 0.746812], ['PRI_lep_pt', 9, 0.746808], ['PRI_lep_pt', 10, 0.746804], ['PRI_lep_pt', 11, 0.746808], ['PRI_lep_pt', 12, 0.7468239999999999], ['PRI_lep_pt', 13, 0.746812], ['PRI_lep_pt', 14, 0.7468159999999999], ['PRI_lep_pt', 15, 0.7468159999999999], ['PRI_lep_pt', 16, 0.7468199999999999], ['PRI_lep_pt', 17, 0.7468199999999999], ['PRI_lep_pt', 18, 0.7468119999999999], ['PRI_lep_pt', 19, 0.7468119999999999], ['PRI_lep_pt', 20, 0.7468079999999999], ['PRI_lep_eta', 2, 0.750812], ['PRI_lep_eta', 3, 0.746908], ['PRI_lep_eta', 4, 0.75014], ['PRI_lep_eta', 5, 0.746884], ['PRI_lep_eta', 6, 0.749484], ['PRI_lep_eta', 7, 0.74686], ['PRI_lep_eta', 8, 0.749032], ['PRI_lep_eta', 9, 0.7468680000000001], ['PRI_lep_eta', 10, 0.748676], ['PRI_lep_eta', 11, 0.746848], ['PRI_lep_eta', 12, 0.7486440000000001], ['PRI_lep_eta', 13, 0.7468359999999999], ['PRI_lep_eta', 14, 0.7482840000000001], ['PRI_lep_eta', 15, 0.746896], ['PRI_lep_eta', 16, 0.7481120000000001], ['PRI_lep_eta', 17, 0.7469199999999999], ['PRI_lep_eta', 18, 0.7479119999999999], ['PRI_lep_eta', 19, 0.7469199999999999], ['PRI_lep_eta', 20, 0.7477880000000001], ['PRI_lep_phi', 2, 0.7468720000000001], ['PRI_lep_phi', 3, 0.7468160000000001], ['PRI_lep_phi', 4, 0.7468679999999999], ['PRI_lep_phi', 5, 0.7468480000000001], ['PRI_lep_phi', 6, 0.746868], ['PRI_lep_phi', 7, 0.746872], ['PRI_lep_phi', 8, 0.746876], ['PRI_lep_phi', 9, 0.7468879999999999], ['PRI_lep_phi', 10, 0.746868], ['PRI_lep_phi', 11, 0.7469319999999999], ['PRI_lep_phi', 12, 0.7468720000000001], ['PRI_lep_phi', 13, 0.746948], ['PRI_lep_phi', 14, 0.7468760000000001], ['PRI_lep_phi', 15, 0.7469720000000001], ['PRI_lep_phi', 16, 0.7469], ['PRI_lep_phi', 17, 0.74696], ['PRI_lep_phi', 18, 0.746892], ['PRI_lep_phi', 19, 0.7469680000000001], ['PRI_lep_phi', 20, 0.74688], ['PRI_met', 2, 0.746872], ['PRI_met', 3, 0.7469319999999999], ['PRI_met', 4, 0.74692], ['PRI_met', 5, 0.7469319999999999], ['PRI_met', 6, 0.7469319999999999], ['PRI_met', 7, 0.7469319999999999], ['PRI_met', 8, 0.7469319999999999], ['PRI_met', 9, 0.7469319999999999], ['PRI_met', 10, 0.7469319999999999], ['PRI_met', 11, 0.7469319999999999], ['PRI_met', 12, 0.7469319999999999], ['PRI_met', 13, 0.7469319999999999], ['PRI_met', 14, 0.7469319999999999], ['PRI_met', 15, 0.7469319999999999], ['PRI_met', 16, 0.7469319999999999], ['PRI_met', 17, 0.7469319999999999], ['PRI_met', 18, 0.7469319999999999], ['PRI_met', 19, 0.7469319999999999], ['PRI_met', 20, 0.7469319999999999], ['PRI_met_phi', 2, 0.7468480000000001], ['PRI_met_phi', 3, 0.746852], ['PRI_met_phi', 4, 0.7468400000000001], ['PRI_met_phi', 5, 0.746864], ['PRI_met_phi', 6, 0.7468199999999999], ['PRI_met_phi', 7, 0.7468960000000001], ['PRI_met_phi', 8, 0.746828], ['PRI_met_phi', 9, 0.7469080000000001], ['PRI_met_phi', 10, 0.7468440000000001], ['PRI_met_phi', 11, 0.7469279999999999], ['PRI_met_phi', 12, 0.746836], ['PRI_met_phi', 13, 0.74694], ['PRI_met_phi', 14, 0.74684], ['PRI_met_phi', 15, 0.7469159999999999], ['PRI_met_phi', 16, 0.7468480000000001], ['PRI_met_phi', 17, 0.7469359999999999], ['PRI_met_phi', 18, 0.746828], ['PRI_met_phi', 19, 0.74696], ['PRI_met_phi', 20, 0.746844], ['PRI_met_sumet', 2, 0.750644], ['PRI_met_sumet', 3, 0.7477560000000001], ['PRI_met_sumet', 4, 0.7472359999999998], ['PRI_met_sumet', 5, 0.7469319999999999], ['PRI_met_sumet', 6, 0.74692], ['PRI_met_sumet', 7, 0.7468959999999999], ['PRI_met_sumet', 8, 0.7468920000000001], ['PRI_met_sumet', 9, 0.7468800000000001], ['PRI_met_sumet', 10, 0.7468800000000001], ['PRI_met_sumet', 11, 0.746868], ['PRI_met_sumet', 12, 0.746868], ['PRI_met_sumet', 13, 0.746872], ['PRI_met_sumet', 14, 0.746872], ['PRI_met_sumet', 15, 0.746872], ['PRI_met_sumet', 16, 0.746872], ['PRI_met_sumet', 17, 0.746868], ['PRI_met_sumet', 18, 0.7468640000000001], ['PRI_met_sumet', 19, 0.7468640000000001], ['PRI_met_sumet', 20, 0.7468640000000001], ['PRI_jet_leading_pt', 2, 0.750064], ['PRI_jet_leading_pt', 3, 0.749484], ['PRI_jet_leading_pt', 4, 0.748648], ['PRI_jet_leading_pt', 5, 0.7480559999999999], ['PRI_jet_leading_pt', 6, 0.7476640000000001], ['PRI_jet_leading_pt', 7, 0.747268], ['PRI_jet_leading_pt', 8, 0.747028], ['PRI_jet_leading_pt', 9, 0.7470199999999999], ['PRI_jet_leading_pt', 10, 0.7470000000000001], ['PRI_jet_leading_pt', 11, 0.7469640000000001], ['PRI_jet_leading_pt', 12, 0.746944], ['PRI_jet_leading_pt', 13, 0.746924], ['PRI_jet_leading_pt', 14, 0.7469239999999999], ['PRI_jet_leading_pt', 15, 0.746908], ['PRI_jet_leading_pt', 16, 0.736496], ['PRI_jet_leading_pt', 17, 0.746904], ['PRI_jet_leading_pt', 18, 0.746908], ['PRI_jet_leading_pt', 19, 0.7468999999999999], ['PRI_jet_leading_pt', 20, 0.7468920000000001], ['PRI_jet_leading_eta', 2, 0.751592], ['PRI_jet_leading_eta', 3, 0.7515959999999999], ['PRI_jet_leading_eta', 4, 0.7516], ['PRI_jet_leading_eta', 5, 0.7516], ['PRI_jet_leading_eta', 6, 0.751612], ['PRI_jet_leading_eta', 7, 0.7516360000000001], ['PRI_jet_leading_eta', 8, 0.751648], ['PRI_jet_leading_eta', 9, 0.7516559999999999], ['PRI_jet_leading_eta', 10, 0.7516839999999998], ['PRI_jet_leading_eta', 11, 0.7516360000000001], ['PRI_jet_leading_eta', 12, 0.75162], ['PRI_jet_leading_eta', 13, 0.7516160000000001], ['PRI_jet_leading_eta', 14, 0.751612], ['PRI_jet_leading_eta', 15, 0.7515999999999999], ['PRI_jet_leading_eta', 16, 0.751604], ['PRI_jet_leading_eta', 17, 0.7515959999999999], ['PRI_jet_leading_eta', 18, 0.751596], ['PRI_jet_leading_eta', 19, 0.751572], ['PRI_jet_leading_eta', 20, 0.751588], ['PRI_jet_leading_phi', 2, 0.74694], ['PRI_jet_leading_phi', 3, 0.74694], ['PRI_jet_leading_phi', 4, 0.7469319999999999], ['PRI_jet_leading_phi', 5, 0.746924], ['PRI_jet_leading_phi', 6, 0.746928], ['PRI_jet_leading_phi', 7, 0.746928], ['PRI_jet_leading_phi', 8, 0.746932], ['PRI_jet_leading_phi', 9, 0.7469280000000001], ['PRI_jet_leading_phi', 10, 0.746928], ['PRI_jet_leading_phi', 11, 0.7469240000000001], ['PRI_jet_leading_phi', 12, 0.7469240000000001], ['PRI_jet_leading_phi', 13, 0.74692], ['PRI_jet_leading_phi', 14, 0.74692], ['PRI_jet_leading_phi', 15, 0.746912], ['PRI_jet_leading_phi', 16, 0.746912], ['PRI_jet_leading_phi', 17, 0.746908], ['PRI_jet_leading_phi', 18, 0.746912], ['PRI_jet_leading_phi', 19, 0.746908], ['PRI_jet_leading_phi', 20, 0.746908], ['PRI_jet_subleading_pt', 2, 0.74698], ['PRI_jet_subleading_pt', 3, 0.746928], ['PRI_jet_subleading_pt', 4, 0.746924], ['PRI_jet_subleading_pt', 5, 0.746888], ['PRI_jet_subleading_pt', 6, 0.746892], ['PRI_jet_subleading_pt', 7, 0.746932], ['PRI_jet_subleading_pt', 8, 0.746924], ['PRI_jet_subleading_pt', 9, 0.746896], ['PRI_jet_subleading_pt', 10, 0.746888], ['PRI_jet_subleading_pt', 11, 0.74688], ['PRI_jet_subleading_pt', 12, 0.746872], ['PRI_jet_subleading_pt', 13, 0.7468680000000001], ['PRI_jet_subleading_pt', 14, 0.7468800000000001], ['PRI_jet_subleading_pt', 15, 0.7468760000000001], ['PRI_jet_subleading_pt', 16, 0.7468760000000001], ['PRI_jet_subleading_pt', 17, 0.74688], ['PRI_jet_subleading_pt', 18, 0.7468839999999999], ['PRI_jet_subleading_pt', 19, 0.746876], ['PRI_jet_subleading_pt', 20, 0.74688], ['PRI_jet_subleading_eta', 2, 0.7469119999999999], ['PRI_jet_subleading_eta', 3, 0.746904], ['PRI_jet_subleading_eta', 4, 0.7469000000000001], ['PRI_jet_subleading_eta', 5, 0.7469000000000001], ['PRI_jet_subleading_eta', 6, 0.7468960000000001], ['PRI_jet_subleading_eta', 7, 0.746888], ['PRI_jet_subleading_eta', 8, 0.746884], ['PRI_jet_subleading_eta', 9, 0.746888], ['PRI_jet_subleading_eta', 10, 0.746884], ['PRI_jet_subleading_eta', 11, 0.7468919999999999], ['PRI_jet_subleading_eta', 12, 0.746888], ['PRI_jet_subleading_eta', 13, 0.746884], ['PRI_jet_subleading_eta', 14, 0.746884], ['PRI_jet_subleading_eta', 15, 0.746884], ['PRI_jet_subleading_eta', 16, 0.746888], ['PRI_jet_subleading_eta', 17, 0.746888], ['PRI_jet_subleading_eta', 18, 0.7468920000000001], ['PRI_jet_subleading_eta', 19, 0.74688], ['PRI_jet_subleading_eta', 20, 0.74688], ['PRI_jet_subleading_phi', 2, 0.746828], ['PRI_jet_subleading_phi', 3, 0.746828], ['PRI_jet_subleading_phi', 4, 0.746828], ['PRI_jet_subleading_phi', 5, 0.746828], ['PRI_jet_subleading_phi', 6, 0.746828], ['PRI_jet_subleading_phi', 7, 0.746828], ['PRI_jet_subleading_phi', 8, 0.746828], ['PRI_jet_subleading_phi', 9, 0.746828], ['PRI_jet_subleading_phi', 10, 0.746828], ['PRI_jet_subleading_phi', 11, 0.746836], ['PRI_jet_subleading_phi', 12, 0.7468400000000001], ['PRI_jet_subleading_phi', 13, 0.7468400000000001], ['PRI_jet_subleading_phi', 14, 0.7468400000000001], ['PRI_jet_subleading_phi', 15, 0.746836], ['PRI_jet_subleading_phi', 16, 0.746836], ['PRI_jet_subleading_phi', 17, 0.7468400000000001], ['PRI_jet_subleading_phi', 18, 0.7468400000000001], ['PRI_jet_subleading_phi', 19, 0.746836], ['PRI_jet_subleading_phi', 20, 0.746836], ['PRI_jet_all_pt', 2, 0.748204], ['PRI_jet_all_pt', 3, 0.747156], ['PRI_jet_all_pt', 4, 0.746904], ['PRI_jet_all_pt', 5, 0.7468960000000001], ['PRI_jet_all_pt', 6, 0.746876], ['PRI_jet_all_pt', 7, 0.7468839999999999], ['PRI_jet_all_pt', 8, 0.7468839999999999], ['PRI_jet_all_pt', 9, 0.746892], ['PRI_jet_all_pt', 10, 0.74688], ['PRI_jet_all_pt', 11, 0.746876], ['PRI_jet_all_pt', 12, 0.746868], ['PRI_jet_all_pt', 13, 0.746868], ['PRI_jet_all_pt', 14, 0.746868], ['PRI_jet_all_pt', 15, 0.746868], ['PRI_jet_all_pt', 16, 0.746872], ['PRI_jet_all_pt', 17, 0.746872], ['PRI_jet_all_pt', 18, 0.746872], ['PRI_jet_all_pt', 19, 0.746872], ['PRI_jet_all_pt', 20, 0.746872]]
best_single_degrees = list(map(lambda l: [l[0], l[1], 0] if 'PRI_jet_leading_eta' in l[0] and l[1] != 10 else l, best_single_degrees))
taken_degrees = list(map(lambda x: (x[0], x[1]), sorted(best_single_degrees, key=lambda x: -x[2])[:30]))

In [ ]:
best_degrees_6 = {'DER_mass_MMC': (6, 0.7763559999999999), 'DER_mass_transverse_met_lep': (6, 0.7506), 'DER_mass_vis': (6, 0.769896), 'DER_pt_h': (6, 0.7518119999999999), 'DER_deltaeta_jet_jet': (2, 0.7482639999999999), 'DER_mass_jet_jet': (6, 0.7479199999999999), 'DER_prodeta_jet_jet': (3, 0.747492), 'DER_deltar_tau_lep': (6, 0.75438), 'DER_pt_tot': (3, 0.7477039999999999), 'DER_sum_pt': (6, 0.7557879999999999), 'DER_pt_ratio_lep_tau': (6, 0.7520640000000001), 'DER_met_phi_centrality': (3, 0.747072), 'DER_lep_eta_centrality': (3, 0.746948), 'PRI_tau_pt': (6, 0.76242), 'PRI_tau_eta': (6, 0.7496120000000001), 'PRI_tau_phi': (4, 0.7470040000000001), 'PRI_lep_pt': (5, 0.747892), 'PRI_lep_eta': (3, 0.750844), 'PRI_lep_phi': (2, 0.7468720000000001), 'PRI_met': (3, 0.746992), 'PRI_met_phi': (1, 0.7468640000000001), 'PRI_met_sumet': (6, 0.752652), 'PRI_jet_leading_pt': (6, 0.752136), 'PRI_jet_leading_eta': (6, 0.7516120000000001), 'PRI_jet_leading_phi': (2, 0.74694), 'PRI_jet_subleading_pt': (5, 0.747292), 'PRI_jet_subleading_eta': (4, 0.74692), 'PRI_jet_subleading_phi': (1, 0.7468640000000001), 'PRI_jet_all_pt': (6, 0.7531000000000001)}
best_degrees_10 = {'DER_mass_MMC': (9, 0.7919720000000001), 'DER_mass_transverse_met_lep': (8, 0.7524120000000001), 'DER_mass_vis': (9, 0.7776440000000001), 'DER_pt_h': (8, 0.753052), 'DER_deltaeta_jet_jet': (8, 0.748804), 'DER_mass_jet_jet': (9, 0.7483359999999999), 'DER_prodeta_jet_jet': (3, 0.747492), 'DER_deltar_tau_lep': (9, 0.756196), 'DER_pt_tot': (8, 0.7477560000000001), 'DER_sum_pt': (9, 0.7558640000000001), 'DER_pt_ratio_lep_tau': (9, 0.7522880000000001), 'DER_met_phi_centrality': (3, 0.747072), 'DER_lep_eta_centrality': (3, 0.746948), 'PRI_tau_pt': (9, 0.7627200000000001), 'PRI_tau_eta': (6, 0.7496120000000001), 'PRI_tau_phi': (4, 0.7470040000000001), 'PRI_lep_pt': (7, 0.747964), 'PRI_lep_eta': (3, 0.750844), 'PRI_lep_phi': (2, 0.7468720000000001), 'PRI_met': (9, 0.747516), 'PRI_met_phi': (1, 0.7468640000000001), 'PRI_met_sumet': (6, 0.752652), 'PRI_jet_leading_pt': (9, 0.752248), 'PRI_jet_leading_eta': (6, 0.7516120000000001), 'PRI_jet_leading_phi': (2, 0.74694), 'PRI_jet_subleading_pt': (5, 0.747292), 'PRI_jet_subleading_eta': (4, 0.74692), 'PRI_jet_subleading_phi': (7, 0.7469520000000001), 'PRI_jet_all_pt': (9, 0.753128)}
best_degrees_13 = {'DER_mass_MMC': (11, 0.8020719999999999), 'DER_mass_transverse_met_lep': (12, 0.752572), 'DER_mass_vis': (11, 0.783824), 'DER_pt_h': (12, 0.7534399999999999), 'DER_deltaeta_jet_jet': (11, 0.748812), 'DER_mass_jet_jet': (12, 0.7484080000000001), 'DER_prodeta_jet_jet': (12, 0.74758), 'DER_deltar_tau_lep': (12, 0.756744), 'DER_pt_tot': (8, 0.7477560000000001), 'DER_sum_pt': (12, 0.7560079999999999), 'DER_pt_ratio_lep_tau': (12, 0.752788), 'DER_met_phi_centrality': (3, 0.747072), 'DER_lep_eta_centrality': (3, 0.746948), 'PRI_tau_pt': (11, 0.762764), 'PRI_tau_eta': (6, 0.7496120000000001), 'PRI_tau_phi': (11, 0.747028), 'PRI_lep_pt': (7, 0.747964), 'PRI_lep_eta': (10, 0.75086), 'PRI_lep_phi': (2, 0.7468720000000001), 'PRI_met': (9, 0.747516), 'PRI_met_phi': (1, 0.7468640000000001), 'PRI_met_sumet': (11, 0.7527519999999999), 'PRI_jet_leading_pt': (9, 0.752248), 'PRI_jet_leading_eta': (11, 0.751684), 'PRI_jet_leading_phi': (2, 0.74694), 'PRI_jet_subleading_pt': (5, 0.747292), 'PRI_jet_subleading_eta': (4, 0.74692), 'PRI_jet_subleading_phi': (7, 0.7469520000000001), 'PRI_jet_all_pt': (10, 0.753144)}

best_degrees = best_degrees_13
best_degrees = {key: best_degrees[key][0] for key in best_degrees if best_degrees[key][1] > 0.766}
best_degrees

In [ ]:
for key in col_to_index_mapping:
  if 'PRI_jet_num' not in key:
      col_index = col_to_index_mapping[key]
      if col_index > col_to_index_mapping['PRI_jet_num']:
        col_index -= 1
      
      tx = np.copy(x) # Recommended to copy x so it doesn't change
      ty = np.copy(y)

      missing_values = tx == -999.

      tx = apply_transformation(tx, [col_to_index_mapping[key] for key in col_to_index_mapping if 'PRI_jet_num' not in key], standardize)
      tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
      tx = apply_transformation(tx, [col_to_index_mapping['PRI_jet_num']], one_hot) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

      tx = apply_transformation(tx, [col_index], lambda x: np.append(x, np.exp(x), axis=1))
      #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
      #tx = tx[valid_idx]
      #ty = ty[valid_idx]

      tx = prepend_bias_column(tx)
      accs, _ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)
      mean_acc = np.mean(accs)


In [ ]:
import copy

#best_degrees = {'DER_mass_MMC': 6, 'DER_mass_transverse_met_lep': 6, 'DER_mass_vis': 6, 'DER_pt_h': 6, 'DER_deltaeta_jet_jet': 2, 'DER_mass_jet_jet': 6, 'DER_prodeta_jet_jet': 3, 'DER_deltar_tau_lep': 6, 'DER_pt_tot': 3, 'DER_sum_pt': 6, 'DER_pt_ratio_lep_tau': 6, 'DER_met_phi_centrality': 3, 'DER_lep_eta_centrality': 3, 'PRI_tau_pt': 6, 'PRI_tau_eta': 6, 'PRI_tau_phi': 4, 'PRI_lep_pt': 5, 'PRI_lep_eta': 3, 'PRI_lep_phi': 2, 'PRI_met': 3, 'PRI_met_phi': 1, 'PRI_met_sumet': 6, 'PRI_jet_leading_pt': 6, 'PRI_jet_leading_eta': 6, 'PRI_jet_leading_phi': 2, 'PRI_jet_subleading_pt': 5, 'PRI_jet_subleading_eta': 4, 'PRI_jet_subleading_phi': 1, 'PRI_jet_all_pt': 6}

min_pairs = []

# This is used

def prepare_data(x, y):
  tx = np.copy(x) # Recommended to copy x so it doesn't change
  ty = np.copy(y)

  col_to_index_mapping_copy = copy.deepcopy(col_to_index_mapping)

  missing_values = tx == -999.

  tx = apply_transformation(tx, [col_to_index_mapping_copy[key] for key in col_to_index_mapping_copy if 'PRI_jet_num' not in key], standardize, col_to_index_mapping_copy)
  tx = nullify_missing_values(tx, missing_values) # standardize and normalize may change value of fields from default missing values, so it uses matrix calculated before applying transformations
  tx = apply_transformation(tx, [col_to_index_mapping_copy['PRI_jet_num']], one_hot, col_to_index_mapping_copy) # recommended to use function that map input to output of same dimensions first (standardize, normalize, etc.) and then functions such as one_hot

  for col_namee in ['PRI_jet_leading_eta', 'DER_pt_ratio_lep_tau', 'PRI_lep_eta', 'PRI_tau_eta']:
    single_col = np.cos(tx[:, [col_to_index_mapping_copy[col_namee]]])
    tx = np.append(tx, single_col, axis=1)

  for col_namee in ['PRI_met_sumet', 'PRI_jet_all_pt']:
    single_col = np.sin(tx[:, [col_to_index_mapping_copy[col_namee]]])
    tx = np.append(tx, single_col, axis=1)

  for col_namee in ['DER_mass_vis', 'PRI_tau_pt']:
    single_col = np.abs(tx[:, [col_to_index_mapping_copy[col_namee]]])
    tx = np.append(tx, single_col, axis=1)

  for pair in ['PRI_tau_pt,PRI_jet_all_pt']:
    k1, k2 = pair.split(',')
    col1 = tx[:, [col_to_index_mapping_copy[k1]]]
    col2 = tx[:, [col_to_index_mapping_copy[k2]]]
    tx = np.append(tx, col1 + col2, axis=1)

  for pair in ['DER_deltar_tau_lep,PRI_tau_pt', 'DER_deltar_tau_lep,PRI_jet_leading_pt', 'DER_deltar_tau_lep,DER_sum_pt']:
    k1, k2 = pair.split(',')
    col1 = tx[:, [col_to_index_mapping_copy[k1]]]
    col2 = tx[:, [col_to_index_mapping_copy[k2]]]
    tx = np.append(tx, col1 * col2, axis=1)

  for pair in min_pairs:#['PRI_tau_pt,PRI_jet_subleading_eta']:
    k1, k2 = pair.split(',')
    col1 = tx[:, [col_to_index_mapping_copy[k1]]]
    col2 = tx[:, [col_to_index_mapping_copy[k2]]]
    tx = np.append(tx, col1 - col2, axis=1)

  for attr, degree in best_degrees.items():
    tx = apply_transformation(tx, [col_to_index_mapping_copy[attr]], lambda x: build_poly(x, degree), col_to_index_mapping_copy)
  #for attr, degree in taken_degrees:
  #  tx_col = tx[:, [col_to_index_mapping_copy[attr]]] ** degree
  #  tx = np.append(tx, tx_col, axis=1)
  #tx = apply_transformation(tx, [col_to_index_mapping_copy['PRI_tau_pt']], lambda x: build_poly(x, 6), col_to_index_mapping_copy)
  #tx = apply_transformation(tx, [col_to_index_mapping_copy['DER_mass_vis']], lambda x: build_poly(x, 6), col_to_index_mapping_copy)
  #tx = apply_transformation(tx, [col_to_index_mapping_copy['DER_mass_MMC']], lambda x: build_poly(x, 6), col_to_index_mapping_copy)

  #tx = apply_transformation(tx, [col_index], lambda x: build_poly(x, degree))
  #valid_idx = np.where(np.sum(missing_values, axis=1) < 9)[0]
  #tx = tx[valid_idx]
  #ty = ty[valid_idx]

  tx = prepend_bias_column(tx)
  return tx, ty

def train_model(y, x):
  w, _ = least_squares(y, x)
  #w, loss = least_squares(y, x)
  def predict(features):
    result = features @ w
    result[np.where(result < 0)] = -1
    result[np.where(result >= 0)] = 1
    return result
  return predict
tx, ty = prepare_data(x, y)
acc, _ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)
mean_acc  = np.mean(acc)
last_acc = mean_acc
for comb in combs:
  min_pairs.append(comb)
  tx, ty = prepare_data(x, y)
  try:
    acc, _ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)
    mean_acc = np.mean(acc)
    if last_acc >= mean_acc:
      min_pairs.pop()
    else:
      last_acc = mean_acc
  except:
    pass

In [ ]:
min_pairs

In [ ]:
last_acc

In [ ]:
d = {'DER_mass_MMC,DER_mass_transverse_met_lep': 0.745404, 'DER_mass_MMC,DER_mass_vis': 0.777992, 'DER_mass_MMC,DER_pt_h': 0.747372, 'DER_mass_MMC,DER_deltaeta_jet_jet': 0.7468640000000001, 'DER_mass_MMC,DER_mass_jet_jet': 0.7468640000000001, 'DER_mass_MMC,DER_prodeta_jet_jet': 0.7468640000000001, 'DER_mass_MMC,DER_deltar_tau_lep': 0.75034, 'DER_mass_MMC,DER_pt_tot': 0.747136, 'DER_mass_MMC,DER_sum_pt': 0.753956, 'DER_mass_MMC,DER_pt_ratio_lep_tau': 0.7527440000000001, 'DER_mass_MMC,DER_met_phi_centrality': 0.746808, 'DER_mass_MMC,DER_lep_eta_centrality': 0.7468640000000001, 'DER_mass_MMC,PRI_tau_pt': 0.7611320000000001, 'DER_mass_MMC,PRI_tau_eta': 0.7493679999999999, 'DER_mass_MMC,PRI_tau_phi': 0.746988, 'DER_mass_MMC,PRI_lep_pt': 0.747748, 'DER_mass_MMC,PRI_lep_eta': 0.750612, 'DER_mass_MMC,PRI_lep_phi': 0.7468640000000001, 'DER_mass_MMC,PRI_met': 0.7463919999999999, 'DER_mass_MMC,PRI_met_phi': 0.7468600000000001, 'DER_mass_MMC,PRI_met_sumet': 0.7508079999999999, 'DER_mass_MMC,PRI_jet_leading_pt': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_leading_eta': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_leading_phi': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_subleading_pt': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_subleading_eta': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_subleading_phi': 0.7468640000000001, 'DER_mass_MMC,PRI_jet_all_pt': 0.7519, 'DER_mass_transverse_met_lep,DER_mass_MMC': 0.745404, 'DER_mass_transverse_met_lep,DER_mass_vis': 0.7586280000000001, 'DER_mass_transverse_met_lep,DER_pt_h': 0.746008, 'DER_mass_transverse_met_lep,DER_deltaeta_jet_jet': 0.745404, 'DER_mass_transverse_met_lep,DER_mass_jet_jet': 0.745404, 'DER_mass_transverse_met_lep,DER_prodeta_jet_jet': 0.745404, 'DER_mass_transverse_met_lep,DER_deltar_tau_lep': 0.7489480000000001, 'DER_mass_transverse_met_lep,DER_pt_tot': 0.7459800000000001, 'DER_mass_transverse_met_lep,DER_sum_pt': 0.7476119999999999, 'DER_mass_transverse_met_lep,DER_pt_ratio_lep_tau': 0.7484919999999999, 'DER_mass_transverse_met_lep,DER_met_phi_centrality': 0.7460160000000001, 'DER_mass_transverse_met_lep,DER_lep_eta_centrality': 0.745404, 'DER_mass_transverse_met_lep,PRI_tau_pt': 0.75024, 'DER_mass_transverse_met_lep,PRI_tau_eta': 0.7478240000000002, 'DER_mass_transverse_met_lep,PRI_tau_phi': 0.7466079999999999, 'DER_mass_transverse_met_lep,PRI_lep_pt': 0.747416, 'DER_mass_transverse_met_lep,PRI_lep_eta': 0.748524, 'DER_mass_transverse_met_lep,PRI_lep_phi': 0.746552, 'DER_mass_transverse_met_lep,PRI_met': 0.7472719999999999, 'DER_mass_transverse_met_lep,PRI_met_phi': 0.7462519999999999, 'DER_mass_transverse_met_lep,PRI_met_sumet': 0.747668, 'DER_mass_transverse_met_lep,PRI_jet_leading_pt': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_leading_eta': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_leading_phi': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_subleading_pt': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_subleading_eta': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_subleading_phi': 0.745404, 'DER_mass_transverse_met_lep,PRI_jet_all_pt': 0.7457840000000001, 'DER_mass_vis,DER_mass_MMC': 0.777992, 'DER_mass_vis,DER_mass_transverse_met_lep': 0.7586280000000001, 'DER_mass_vis,DER_pt_h': 0.766672, 'DER_mass_vis,DER_deltaeta_jet_jet': 0.777992, 'DER_mass_vis,DER_mass_jet_jet': 0.777992, 'DER_mass_vis,DER_prodeta_jet_jet': 0.777992, 'DER_mass_vis,DER_deltar_tau_lep': 0.75324, 'DER_mass_vis,DER_pt_tot': 0.763496, 'DER_mass_vis,DER_sum_pt': 0.760972, 'DER_mass_vis,DER_pt_ratio_lep_tau': 0.7724559999999999, 'DER_mass_vis,DER_met_phi_centrality': 0.7728879999999999, 'DER_mass_vis,DER_lep_eta_centrality': 0.777992, 'DER_mass_vis,PRI_tau_pt': 0.753788, 'DER_mass_vis,PRI_tau_eta': 0.7513119999999999, 'DER_mass_vis,PRI_tau_phi': 0.7581119999999999, 'DER_mass_vis,PRI_lep_pt': 0.758328, 'DER_mass_vis,PRI_lep_eta': 0.7508800000000001, 'DER_mass_vis,PRI_lep_phi': 0.75912, 'DER_mass_vis,PRI_met': 0.761852, 'DER_mass_vis,PRI_met_phi': 0.7589480000000001, 'DER_mass_vis,PRI_met_sumet': 0.75556, 'DER_mass_vis,PRI_jet_leading_pt': 0.777992, 'DER_mass_vis,PRI_jet_leading_eta': 0.777992, 'DER_mass_vis,PRI_jet_leading_phi': 0.777992, 'DER_mass_vis,PRI_jet_subleading_pt': 0.777992, 'DER_mass_vis,PRI_jet_subleading_eta': 0.777992, 'DER_mass_vis,PRI_jet_subleading_phi': 0.777992, 'DER_mass_vis,PRI_jet_all_pt': 0.768128, 'DER_pt_h,DER_mass_MMC': 0.747372, 'DER_pt_h,DER_mass_transverse_met_lep': 0.746008, 'DER_pt_h,DER_mass_vis': 0.766672, 'DER_pt_h,DER_deltaeta_jet_jet': 0.747372, 'DER_pt_h,DER_mass_jet_jet': 0.747372, 'DER_pt_h,DER_prodeta_jet_jet': 0.747372, 'DER_pt_h,DER_deltar_tau_lep': 0.7488080000000001, 'DER_pt_h,DER_pt_tot': 0.747892, 'DER_pt_h,DER_sum_pt': 0.749028, 'DER_pt_h,DER_pt_ratio_lep_tau': 0.752828, 'DER_pt_h,DER_met_phi_centrality': 0.747724, 'DER_pt_h,DER_lep_eta_centrality': 0.747372, 'DER_pt_h,PRI_tau_pt': 0.7533399999999999, 'DER_pt_h,PRI_tau_eta': 0.74796, 'DER_pt_h,PRI_tau_phi': 0.747056, 'DER_pt_h,PRI_lep_pt': 0.7471639999999999, 'DER_pt_h,PRI_lep_eta': 0.748724, 'DER_pt_h,PRI_lep_phi': 0.7467600000000001, 'DER_pt_h,PRI_met': 0.746764, 'DER_pt_h,PRI_met_phi': 0.7470959999999999, 'DER_pt_h,PRI_met_sumet': 0.7490479999999999, 'DER_pt_h,PRI_jet_leading_pt': 0.747372, 'DER_pt_h,PRI_jet_leading_eta': 0.747372, 'DER_pt_h,PRI_jet_leading_phi': 0.747372, 'DER_pt_h,PRI_jet_subleading_pt': 0.747372, 'DER_pt_h,PRI_jet_subleading_eta': 0.747372, 'DER_pt_h,PRI_jet_subleading_phi': 0.747372, 'DER_pt_h,PRI_jet_all_pt': 0.7471759999999998, 'DER_deltaeta_jet_jet,DER_mass_MMC': 0.7468640000000001, 'DER_deltaeta_jet_jet,DER_mass_transverse_met_lep': 0.745404, 'DER_deltaeta_jet_jet,DER_mass_vis': 0.777992, 'DER_deltaeta_jet_jet,DER_pt_h': 0.747372, 'DER_deltaeta_jet_jet,DER_mass_jet_jet': 0.7468640000000001, 'DER_deltaeta_jet_jet,DER_prodeta_jet_jet': 0.7468640000000001, 'DER_deltaeta_jet_jet,DER_deltar_tau_lep': 0.75034, 'DER_deltaeta_jet_jet,DER_pt_tot': 0.747136, 'DER_deltaeta_jet_jet,DER_sum_pt': 0.75396, 'DER_deltaeta_jet_jet,DER_pt_ratio_lep_tau': 0.7527440000000001, 'DER_deltaeta_jet_jet,DER_met_phi_centrality': 0.746808, 'DER_deltaeta_jet_jet,DER_lep_eta_centrality': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_tau_pt': 0.7611320000000001, 'DER_deltaeta_jet_jet,PRI_tau_eta': 0.7493679999999999, 'DER_deltaeta_jet_jet,PRI_tau_phi': 0.746988, 'DER_deltaeta_jet_jet,PRI_lep_pt': 0.747748, 'DER_deltaeta_jet_jet,PRI_lep_eta': 0.7506079999999999, 'DER_deltaeta_jet_jet,PRI_lep_phi': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_met': 0.7463919999999999, 'DER_deltaeta_jet_jet,PRI_met_phi': 0.7468600000000001, 'DER_deltaeta_jet_jet,PRI_met_sumet': 0.7508079999999999, 'DER_deltaeta_jet_jet,PRI_jet_leading_pt': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_leading_eta': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_leading_phi': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_subleading_pt': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_subleading_eta': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_subleading_phi': 0.7468640000000001, 'DER_deltaeta_jet_jet,PRI_jet_all_pt': 0.7519, 'DER_mass_jet_jet,DER_mass_MMC': 0.7468640000000001, 'DER_mass_jet_jet,DER_mass_transverse_met_lep': 0.745404, 'DER_mass_jet_jet,DER_mass_vis': 0.777992, 'DER_mass_jet_jet,DER_pt_h': 0.747372, 'DER_mass_jet_jet,DER_deltaeta_jet_jet': 0.7468640000000001, 'DER_mass_jet_jet,DER_prodeta_jet_jet': 0.7468640000000001, 'DER_mass_jet_jet,DER_deltar_tau_lep': 0.750332, 'DER_mass_jet_jet,DER_pt_tot': 0.747136, 'DER_mass_jet_jet,DER_sum_pt': 0.753956, 'DER_mass_jet_jet,DER_pt_ratio_lep_tau': 0.7527440000000001, 'DER_mass_jet_jet,DER_met_phi_centrality': 0.746808, 'DER_mass_jet_jet,DER_lep_eta_centrality': 0.7468640000000001, 'DER_mass_jet_jet,PRI_tau_pt': 0.7611320000000001, 'DER_mass_jet_jet,PRI_tau_eta': 0.7493679999999999, 'DER_mass_jet_jet,PRI_tau_phi': 0.746988, 'DER_mass_jet_jet,PRI_lep_pt': 0.747748, 'DER_mass_jet_jet,PRI_lep_eta': 0.750612, 'DER_mass_jet_jet,PRI_lep_phi': 0.7468640000000001, 'DER_mass_jet_jet,PRI_met': 0.7463919999999999, 'DER_mass_jet_jet,PRI_met_phi': 0.7468600000000001, 'DER_mass_jet_jet,PRI_met_sumet': 0.7508079999999999, 'DER_mass_jet_jet,PRI_jet_leading_pt': 0.7468640000000001, 'DER_mass_jet_jet,PRI_jet_leading_eta': 0.7468640000000001, 'DER_mass_jet_jet,PRI_jet_leading_phi': 0.7468600000000001, 'DER_mass_jet_jet,PRI_jet_subleading_pt': 0.7468640000000001, 'DER_mass_jet_jet,PRI_jet_subleading_eta': 0.7468640000000001, 'DER_mass_jet_jet,PRI_jet_subleading_phi': 0.7468640000000001, 'DER_mass_jet_jet,PRI_jet_all_pt': 0.751896, 'DER_prodeta_jet_jet,DER_mass_MMC': 0.7468640000000001, 'DER_prodeta_jet_jet,DER_mass_transverse_met_lep': 0.745404, 'DER_prodeta_jet_jet,DER_mass_vis': 0.777992, 'DER_prodeta_jet_jet,DER_pt_h': 0.747372, 'DER_prodeta_jet_jet,DER_deltaeta_jet_jet': 0.7468640000000001, 'DER_prodeta_jet_jet,DER_mass_jet_jet': 0.7468640000000001, 'DER_prodeta_jet_jet,DER_deltar_tau_lep': 0.75034, 'DER_prodeta_jet_jet,DER_pt_tot': 0.747136, 'DER_prodeta_jet_jet,DER_sum_pt': 0.75396, 'DER_prodeta_jet_jet,DER_pt_ratio_lep_tau': 0.7527440000000001, 'DER_prodeta_jet_jet,DER_met_phi_centrality': 0.746808, 'DER_prodeta_jet_jet,DER_lep_eta_centrality': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_tau_pt': 0.7611320000000001, 'DER_prodeta_jet_jet,PRI_tau_eta': 0.7493679999999999, 'DER_prodeta_jet_jet,PRI_tau_phi': 0.746988, 'DER_prodeta_jet_jet,PRI_lep_pt': 0.747748, 'DER_prodeta_jet_jet,PRI_lep_eta': 0.750612, 'DER_prodeta_jet_jet,PRI_lep_phi': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_met': 0.7463919999999999, 'DER_prodeta_jet_jet,PRI_met_phi': 0.7468600000000001, 'DER_prodeta_jet_jet,PRI_met_sumet': 0.7508079999999999, 'DER_prodeta_jet_jet,PRI_jet_leading_pt': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_jet_leading_eta': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_jet_leading_phi': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_jet_subleading_pt': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_jet_subleading_eta': 0.7468640000000001, 
    'DER_prodeta_jet_jet,PRI_jet_subleading_phi': 0.7468640000000001, 'DER_prodeta_jet_jet,PRI_jet_all_pt': 0.7519, 'DER_deltar_tau_lep,DER_mass_MMC': 0.75034, 'DER_deltar_tau_lep,DER_mass_transverse_met_lep': 0.7489480000000001, 'DER_deltar_tau_lep,DER_mass_vis': 0.75324, 'DER_deltar_tau_lep,DER_pt_h': 0.7488080000000001, 'DER_deltar_tau_lep,DER_deltaeta_jet_jet': 0.75034, 'DER_deltar_tau_lep,DER_mass_jet_jet': 0.750332, 'DER_deltar_tau_lep,DER_prodeta_jet_jet': 0.75034, 'DER_deltar_tau_lep,DER_pt_tot': 0.750028, 'DER_deltar_tau_lep,DER_sum_pt': 0.7468639999999999, 'DER_deltar_tau_lep,DER_pt_ratio_lep_tau': 0.7573920000000001, 'DER_deltar_tau_lep,DER_met_phi_centrality': 0.7504360000000001, 'DER_deltar_tau_lep,DER_lep_eta_centrality': 0.75034, 'DER_deltar_tau_lep,PRI_tau_pt': 0.747228, 'DER_deltar_tau_lep,PRI_tau_eta': 0.74676, 'DER_deltar_tau_lep,PRI_tau_phi': 0.7482840000000001, 'DER_deltar_tau_lep,PRI_lep_pt': 0.747412, 'DER_deltar_tau_lep,PRI_lep_eta': 0.7469759999999999, 'DER_deltar_tau_lep,PRI_lep_phi': 0.7485160000000001, 'DER_deltar_tau_lep,PRI_met': 0.747788, 'DER_deltar_tau_lep,PRI_met_phi': 0.748756, 'DER_deltar_tau_lep,PRI_met_sumet': 0.7468440000000001, 'DER_deltar_tau_lep,PRI_jet_leading_pt': 0.75034, 'DER_deltar_tau_lep,PRI_jet_leading_eta': 0.75034, 'DER_deltar_tau_lep,PRI_jet_leading_phi': 0.75034, 'DER_deltar_tau_lep,PRI_jet_subleading_pt': 0.75034, 'DER_deltar_tau_lep,PRI_jet_subleading_eta': 0.7503560000000001, 'DER_deltar_tau_lep,PRI_jet_subleading_phi': 0.75034, 'DER_deltar_tau_lep,PRI_jet_all_pt': 0.747824, 'DER_pt_tot,DER_mass_MMC': 0.747136, 'DER_pt_tot,DER_mass_transverse_met_lep': 0.7459800000000001, 'DER_pt_tot,DER_mass_vis': 0.763496, 'DER_pt_tot,DER_pt_h': 0.747892, 'DER_pt_tot,DER_deltaeta_jet_jet': 0.747136, 'DER_pt_tot,DER_mass_jet_jet': 0.747136, 'DER_pt_tot,DER_prodeta_jet_jet': 0.747136, 'DER_pt_tot,DER_deltar_tau_lep': 0.750028, 'DER_pt_tot,DER_sum_pt': 0.7501880000000001, 'DER_pt_tot,DER_pt_ratio_lep_tau': 0.7487440000000001, 'DER_pt_tot,DER_met_phi_centrality': 0.7469840000000001, 'DER_pt_tot,DER_lep_eta_centrality': 0.747136, 'DER_pt_tot,PRI_tau_pt': 0.754028, 'DER_pt_tot,PRI_tau_eta': 0.749536, 'DER_pt_tot,PRI_tau_phi': 0.7473040000000001, 'DER_pt_tot,PRI_lep_pt': 0.7481519999999999, 'DER_pt_tot,PRI_lep_eta': 0.7499480000000001, 'DER_pt_tot,PRI_lep_phi': 0.7472479999999999, 'DER_pt_tot,PRI_met': 0.746648, 'DER_pt_tot,PRI_met_phi': 0.7470480000000002, 'DER_pt_tot,PRI_met_sumet': 0.7507560000000001, 'DER_pt_tot,PRI_jet_leading_pt': 0.747136, 'DER_pt_tot,PRI_jet_leading_eta': 0.747136, 'DER_pt_tot,PRI_jet_leading_phi': 0.747136, 'DER_pt_tot,PRI_jet_subleading_pt': 0.747136, 'DER_pt_tot,PRI_jet_subleading_eta': 0.747136, 'DER_pt_tot,PRI_jet_subleading_phi': 0.747136, 'DER_pt_tot,PRI_jet_all_pt': 0.749128, 'DER_sum_pt,DER_mass_MMC': 0.753956, 'DER_sum_pt,DER_mass_transverse_met_lep': 0.7476119999999999, 'DER_sum_pt,DER_mass_vis': 0.760972, 'DER_sum_pt,DER_pt_h': 0.749028, 'DER_sum_pt,DER_deltaeta_jet_jet': 0.75396, 'DER_sum_pt,DER_mass_jet_jet': 0.753956, 'DER_sum_pt,DER_prodeta_jet_jet': 0.75396, 'DER_sum_pt,DER_deltar_tau_lep': 0.7468639999999999, 'DER_sum_pt,DER_pt_tot': 0.7501880000000001, 'DER_sum_pt,DER_pt_ratio_lep_tau': 0.757272, 'DER_sum_pt,DER_met_phi_centrality': 0.7516200000000001, 'DER_sum_pt,DER_lep_eta_centrality': 0.753956, 'DER_sum_pt,PRI_tau_pt': 0.749368, 'DER_sum_pt,PRI_tau_eta': 0.747052, 'DER_sum_pt,PRI_tau_phi': 0.7479, 'DER_sum_pt,PRI_lep_pt': 0.747048, 'DER_sum_pt,PRI_lep_eta': 0.7470039999999999, 'DER_sum_pt,PRI_lep_phi': 0.7478199999999999, 'DER_sum_pt,PRI_met': 0.748304, 'DER_sum_pt,PRI_met_phi': 0.748236, 'DER_sum_pt,PRI_met_sumet': 0.746972, 'DER_sum_pt,PRI_jet_leading_pt': 0.753956, 'DER_sum_pt,PRI_jet_leading_eta': 0.753948, 'DER_sum_pt,PRI_jet_leading_phi': 0.75396, 'DER_sum_pt,PRI_jet_subleading_pt': 0.75396, 'DER_sum_pt,PRI_jet_subleading_eta': 0.7539960000000001, 'DER_sum_pt,PRI_jet_subleading_phi': 0.753956, 'DER_sum_pt,PRI_jet_all_pt': 0.7493879999999999, 'DER_pt_ratio_lep_tau,DER_mass_MMC': 0.7527440000000001, 'DER_pt_ratio_lep_tau,DER_mass_transverse_met_lep': 0.7484919999999999, 'DER_pt_ratio_lep_tau,DER_mass_vis': 0.7724559999999999, 'DER_pt_ratio_lep_tau,DER_pt_h': 0.752828, 'DER_pt_ratio_lep_tau,DER_deltaeta_jet_jet': 0.7527440000000001, 'DER_pt_ratio_lep_tau,DER_mass_jet_jet': 0.7527440000000001, 'DER_pt_ratio_lep_tau,DER_prodeta_jet_jet': 0.7527440000000001, 'DER_pt_ratio_lep_tau,DER_deltar_tau_lep': 0.7573920000000001, 'DER_pt_ratio_lep_tau,DER_pt_tot': 0.7487440000000001, 'DER_pt_ratio_lep_tau,DER_sum_pt': 0.757272, 'DER_pt_ratio_lep_tau,DER_met_phi_centrality': 0.7501879999999999, 'DER_pt_ratio_lep_tau,DER_lep_eta_centrality': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_tau_pt': 0.757896, 'DER_pt_ratio_lep_tau,PRI_tau_eta': 0.7541279999999999, 'DER_pt_ratio_lep_tau,PRI_tau_phi': 0.7499279999999999, 'DER_pt_ratio_lep_tau,PRI_lep_pt': 0.759956, 'DER_pt_ratio_lep_tau,PRI_lep_eta': 0.7557, 'DER_pt_ratio_lep_tau,PRI_lep_phi': 0.749652, 'DER_pt_ratio_lep_tau,PRI_met': 0.7491199999999999, 'DER_pt_ratio_lep_tau,PRI_met_phi': 0.7494080000000001, 'DER_pt_ratio_lep_tau,PRI_met_sumet': 0.7564200000000001, 'DER_pt_ratio_lep_tau,PRI_jet_leading_pt': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_leading_eta': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_leading_phi': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_subleading_pt': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_subleading_eta': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_subleading_phi': 0.7527440000000001, 'DER_pt_ratio_lep_tau,PRI_jet_all_pt': 0.756, 'DER_met_phi_centrality,DER_mass_MMC': 0.746808, 'DER_met_phi_centrality,DER_mass_transverse_met_lep': 0.7460160000000001, 'DER_met_phi_centrality,DER_mass_vis': 0.7728879999999999, 'DER_met_phi_centrality,DER_pt_h': 0.747724, 'DER_met_phi_centrality,DER_deltaeta_jet_jet': 0.746808, 'DER_met_phi_centrality,DER_mass_jet_jet': 0.746808, 'DER_met_phi_centrality,DER_prodeta_jet_jet': 0.746808, 'DER_met_phi_centrality,DER_deltar_tau_lep': 0.7504360000000001, 'DER_met_phi_centrality,DER_pt_tot': 0.7469840000000001, 'DER_met_phi_centrality,DER_sum_pt': 0.7516200000000001, 'DER_met_phi_centrality,DER_pt_ratio_lep_tau': 0.7501879999999999, 'DER_met_phi_centrality,DER_lep_eta_centrality': 0.746808, 'DER_met_phi_centrality,PRI_tau_pt': 0.757128, 'DER_met_phi_centrality,PRI_tau_eta': 0.7491840000000001, 'DER_met_phi_centrality,PRI_tau_phi': 0.747188, 'DER_met_phi_centrality,PRI_lep_pt': 0.747896, 'DER_met_phi_centrality,PRI_lep_eta': 0.750768, 'DER_met_phi_centrality,PRI_lep_phi': 0.7470439999999999, 'DER_met_phi_centrality,PRI_met': 0.746564, 'DER_met_phi_centrality,PRI_met_phi': 0.74684, 'DER_met_phi_centrality,PRI_met_sumet': 0.750644, 'DER_met_phi_centrality,PRI_jet_leading_pt': 0.746804, 'DER_met_phi_centrality,PRI_jet_leading_eta': 0.746808, 'DER_met_phi_centrality,PRI_jet_leading_phi': 0.746808, 'DER_met_phi_centrality,PRI_jet_subleading_pt': 0.746808, 'DER_met_phi_centrality,PRI_jet_subleading_eta': 0.746808, 'DER_met_phi_centrality,PRI_jet_subleading_phi': 0.746808, 'DER_met_phi_centrality,PRI_jet_all_pt': 0.7489, 'DER_lep_eta_centrality,DER_mass_MMC': 0.7468640000000001, 'DER_lep_eta_centrality,DER_mass_transverse_met_lep': 0.745404, 'DER_lep_eta_centrality,DER_mass_vis': 0.777992, 'DER_lep_eta_centrality,DER_pt_h': 0.747372, 'DER_lep_eta_centrality,DER_deltaeta_jet_jet': 0.7468640000000001, 'DER_lep_eta_centrality,DER_mass_jet_jet': 0.7468640000000001, 'DER_lep_eta_centrality,DER_prodeta_jet_jet': 0.7468640000000001, 'DER_lep_eta_centrality,DER_deltar_tau_lep': 0.75034, 'DER_lep_eta_centrality,DER_pt_tot': 0.747136, 'DER_lep_eta_centrality,DER_sum_pt': 0.753956, 'DER_lep_eta_centrality,DER_pt_ratio_lep_tau': 0.7527440000000001, 'DER_lep_eta_centrality,DER_met_phi_centrality': 0.746808, 'DER_lep_eta_centrality,PRI_tau_pt': 0.7611320000000001, 'DER_lep_eta_centrality,PRI_tau_eta': 0.7493679999999999, 'DER_lep_eta_centrality,PRI_tau_phi': 0.746988, 'DER_lep_eta_centrality,PRI_lep_pt': 0.747748, 'DER_lep_eta_centrality,PRI_lep_eta': 0.750612, 'DER_lep_eta_centrality,PRI_lep_phi': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_met': 0.7463919999999999, 'DER_lep_eta_centrality,PRI_met_phi': 0.7468600000000001, 'DER_lep_eta_centrality,PRI_met_sumet': 0.7508079999999999, 'DER_lep_eta_centrality,PRI_jet_leading_pt': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_leading_eta': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_leading_phi': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_subleading_pt': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_subleading_eta': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_subleading_phi': 0.7468640000000001, 'DER_lep_eta_centrality,PRI_jet_all_pt': 0.7519, 'PRI_tau_pt,DER_mass_MMC': 0.7611320000000001, 'PRI_tau_pt,DER_mass_transverse_met_lep': 0.75024, 'PRI_tau_pt,DER_mass_vis': 0.753788, 'PRI_tau_pt,DER_pt_h': 0.7533399999999999, 'PRI_tau_pt,DER_deltaeta_jet_jet': 0.7611320000000001, 'PRI_tau_pt,DER_mass_jet_jet': 0.7611320000000001, 'PRI_tau_pt,DER_prodeta_jet_jet': 0.7611320000000001, 'PRI_tau_pt,DER_deltar_tau_lep': 0.747228, 'PRI_tau_pt,DER_pt_tot': 0.754028, 'PRI_tau_pt,DER_sum_pt': 0.749368, 'PRI_tau_pt,DER_pt_ratio_lep_tau': 0.757896, 'PRI_tau_pt,DER_met_phi_centrality': 0.757128, 'PRI_tau_pt,DER_lep_eta_centrality': 0.7611320000000001, 'PRI_tau_pt,PRI_tau_eta': 0.7473759999999999, 'PRI_tau_pt,PRI_tau_phi': 0.750068, 'PRI_tau_pt,PRI_lep_pt': 0.748228, 'PRI_tau_pt,PRI_lep_eta': 0.7470600000000001, 'PRI_tau_pt,PRI_lep_phi': 0.75112, 'PRI_tau_pt,PRI_met': 0.751052, 'PRI_tau_pt,PRI_met_phi': 0.750772, 'PRI_tau_pt,PRI_met_sumet': 0.74766, 'PRI_tau_pt,PRI_jet_leading_pt': 0.7611320000000001, 'PRI_tau_pt,PRI_jet_leading_eta': 0.7611320000000001, 'PRI_tau_pt,PRI_jet_leading_phi': 0.7611320000000001, 'PRI_tau_pt,PRI_jet_subleading_pt': 0.7611320000000001, 'PRI_tau_pt,PRI_jet_subleading_eta': 0.7611360000000001, 
    'PRI_tau_pt,PRI_jet_subleading_phi': 0.7611320000000001, 'PRI_tau_pt,PRI_jet_all_pt': 0.753144, 'PRI_tau_eta,DER_mass_MMC': 0.7493679999999999, 'PRI_tau_eta,DER_mass_transverse_met_lep': 0.7478240000000002, 'PRI_tau_eta,DER_mass_vis': 0.7513119999999999, 'PRI_tau_eta,DER_pt_h': 0.74796, 'PRI_tau_eta,DER_deltaeta_jet_jet': 0.7493679999999999, 'PRI_tau_eta,DER_mass_jet_jet': 0.7493679999999999, 'PRI_tau_eta,DER_prodeta_jet_jet': 0.7493679999999999, 'PRI_tau_eta,DER_deltar_tau_lep': 0.74676, 'PRI_tau_eta,DER_pt_tot': 0.749536, 'PRI_tau_eta,DER_sum_pt': 0.747052, 'PRI_tau_eta,DER_pt_ratio_lep_tau': 0.7541279999999999, 'PRI_tau_eta,DER_met_phi_centrality': 0.7491840000000001, 'PRI_tau_eta,DER_lep_eta_centrality': 0.7493679999999999, 'PRI_tau_eta,PRI_tau_pt': 0.7473759999999999, 'PRI_tau_eta,PRI_tau_phi': 0.7475879999999999, 'PRI_tau_eta,PRI_lep_pt': 0.7472199999999999, 'PRI_tau_eta,PRI_lep_eta': 0.746964, 'PRI_tau_eta,PRI_lep_phi': 0.748232, 'PRI_tau_eta,PRI_met': 0.7482599999999999, 'PRI_tau_eta,PRI_met_phi': 0.748504, 'PRI_tau_eta,PRI_met_sumet': 0.7469240000000001, 'PRI_tau_eta,PRI_jet_leading_pt': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_leading_eta': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_leading_phi': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_subleading_pt': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_subleading_eta': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_subleading_phi': 0.7493679999999999, 'PRI_tau_eta,PRI_jet_all_pt': 0.74758, 'PRI_tau_phi,DER_mass_MMC': 0.746988, 'PRI_tau_phi,DER_mass_transverse_met_lep': 0.7466079999999999, 'PRI_tau_phi,DER_mass_vis': 0.7581119999999999, 'PRI_tau_phi,DER_pt_h': 0.747056, 'PRI_tau_phi,DER_deltaeta_jet_jet': 0.746988, 'PRI_tau_phi,DER_mass_jet_jet': 0.746988, 'PRI_tau_phi,DER_prodeta_jet_jet': 0.746988, 'PRI_tau_phi,DER_deltar_tau_lep': 0.7482840000000001, 'PRI_tau_phi,DER_pt_tot': 0.7473040000000001, 'PRI_tau_phi,DER_sum_pt': 0.7479, 'PRI_tau_phi,DER_pt_ratio_lep_tau': 0.7499279999999999, 'PRI_tau_phi,DER_met_phi_centrality': 0.747188, 'PRI_tau_phi,DER_lep_eta_centrality': 0.746988, 'PRI_tau_phi,PRI_tau_pt': 0.750068, 'PRI_tau_phi,PRI_tau_eta': 0.7475879999999999, 'PRI_tau_phi,PRI_lep_pt': 0.7473479999999999, 'PRI_tau_phi,PRI_lep_eta': 0.747944, 'PRI_tau_phi,PRI_lep_phi': 0.746872, 'PRI_tau_phi,PRI_met': 0.746852, 'PRI_tau_phi,PRI_met_phi': 0.746944, 'PRI_tau_phi,PRI_met_sumet': 0.747736, 'PRI_tau_phi,PRI_jet_leading_pt': 0.746988, 'PRI_tau_phi,PRI_jet_leading_eta': 0.746988, 'PRI_tau_phi,PRI_jet_leading_phi': 0.746988, 'PRI_tau_phi,PRI_jet_subleading_pt': 0.746988, 'PRI_tau_phi,PRI_jet_subleading_eta': 0.746988, 'PRI_tau_phi,PRI_jet_subleading_phi': 0.746988, 'PRI_tau_phi,PRI_jet_all_pt': 0.747044, 'PRI_lep_pt,DER_mass_MMC': 0.747748, 'PRI_lep_pt,DER_mass_transverse_met_lep': 0.747416, 'PRI_lep_pt,DER_mass_vis': 0.758328, 'PRI_lep_pt,DER_pt_h': 0.7471639999999999, 'PRI_lep_pt,DER_deltaeta_jet_jet': 0.747748, 'PRI_lep_pt,DER_mass_jet_jet': 0.747748, 'PRI_lep_pt,DER_prodeta_jet_jet': 0.747748, 'PRI_lep_pt,DER_deltar_tau_lep': 0.747412, 'PRI_lep_pt,DER_pt_tot': 0.7481519999999999, 'PRI_lep_pt,DER_sum_pt': 0.747048, 'PRI_lep_pt,DER_pt_ratio_lep_tau': 0.759956, 'PRI_lep_pt,DER_met_phi_centrality': 0.747896, 'PRI_lep_pt,DER_lep_eta_centrality': 0.747748, 'PRI_lep_pt,PRI_tau_pt': 0.748228, 'PRI_lep_pt,PRI_tau_eta': 0.7472199999999999, 'PRI_lep_pt,PRI_tau_phi': 0.7473479999999999, 'PRI_lep_pt,PRI_lep_eta': 0.747492, 'PRI_lep_pt,PRI_lep_phi': 0.747288, 'PRI_lep_pt,PRI_met': 0.74696, 'PRI_lep_pt,PRI_met_phi': 0.7474480000000001, 'PRI_lep_pt,PRI_met_sumet': 0.747116, 'PRI_lep_pt,PRI_jet_leading_pt': 0.747748, 'PRI_lep_pt,PRI_jet_leading_eta': 0.747748, 'PRI_lep_pt,PRI_jet_leading_phi': 0.747748, 'PRI_lep_pt,PRI_jet_subleading_pt': 0.747748, 'PRI_lep_pt,PRI_jet_subleading_eta': 0.747748, 'PRI_lep_pt,PRI_jet_subleading_phi': 0.747748, 'PRI_lep_pt,PRI_jet_all_pt': 0.746904, 'PRI_lep_eta,DER_mass_MMC': 0.750612, 'PRI_lep_eta,DER_mass_transverse_met_lep': 0.748524, 'PRI_lep_eta,DER_mass_vis': 0.7508800000000001, 'PRI_lep_eta,DER_pt_h': 0.748724, 'PRI_lep_eta,DER_deltaeta_jet_jet': 0.7506079999999999, 'PRI_lep_eta,DER_mass_jet_jet': 0.750612, 'PRI_lep_eta,DER_prodeta_jet_jet': 0.750612, 'PRI_lep_eta,DER_deltar_tau_lep': 0.7469759999999999, 'PRI_lep_eta,DER_pt_tot': 0.7499480000000001, 'PRI_lep_eta,DER_sum_pt': 0.7470039999999999, 'PRI_lep_eta,DER_pt_ratio_lep_tau': 0.7557, 'PRI_lep_eta,DER_met_phi_centrality': 0.750768, 'PRI_lep_eta,DER_lep_eta_centrality': 0.750612, 'PRI_lep_eta,PRI_tau_pt': 0.7470600000000001, 'PRI_lep_eta,PRI_tau_eta': 0.746964, 'PRI_lep_eta,PRI_tau_phi': 0.747944, 'PRI_lep_eta,PRI_lep_pt': 0.747492, 'PRI_lep_eta,PRI_lep_phi': 0.7487880000000001, 'PRI_lep_eta,PRI_met': 0.7487119999999999, 'PRI_lep_eta,PRI_met_phi': 0.7487320000000001, 'PRI_lep_eta,PRI_met_sumet': 0.7470160000000001, 'PRI_lep_eta,PRI_jet_leading_pt': 0.750612, 'PRI_lep_eta,PRI_jet_leading_eta': 0.750612, 'PRI_lep_eta,PRI_jet_leading_phi': 0.750612, 'PRI_lep_eta,PRI_jet_subleading_pt': 0.750612, 'PRI_lep_eta,PRI_jet_subleading_eta': 0.750612, 'PRI_lep_eta,PRI_jet_subleading_phi': 0.750612, 'PRI_lep_eta,PRI_jet_all_pt': 0.748336, 'PRI_lep_phi,DER_mass_MMC': 0.7468640000000001, 'PRI_lep_phi,DER_mass_transverse_met_lep': 0.746552, 'PRI_lep_phi,DER_mass_vis': 0.75912, 'PRI_lep_phi,DER_pt_h': 0.7467600000000001, 'PRI_lep_phi,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_lep_phi,DER_mass_jet_jet': 0.7468640000000001, 'PRI_lep_phi,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_lep_phi,DER_deltar_tau_lep': 0.7485160000000001, 'PRI_lep_phi,DER_pt_tot': 0.7472479999999999, 'PRI_lep_phi,DER_sum_pt': 0.7478199999999999, 'PRI_lep_phi,DER_pt_ratio_lep_tau': 0.749652, 'PRI_lep_phi,DER_met_phi_centrality': 0.7470439999999999, 'PRI_lep_phi,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_lep_phi,PRI_tau_pt': 0.75112, 'PRI_lep_phi,PRI_tau_eta': 0.748232, 'PRI_lep_phi,PRI_tau_phi': 0.746872, 'PRI_lep_phi,PRI_lep_pt': 0.747288, 'PRI_lep_phi,PRI_lep_eta': 0.7487880000000001, 'PRI_lep_phi,PRI_met': 0.7465079999999998, 'PRI_lep_phi,PRI_met_phi': 0.7468680000000001, 'PRI_lep_phi,PRI_met_sumet': 0.7479439999999999, 'PRI_lep_phi,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_subleading_pt': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_lep_phi,PRI_jet_all_pt': 0.747084, 'PRI_met,DER_mass_MMC': 0.7463919999999999, 'PRI_met,DER_mass_transverse_met_lep': 0.7472719999999999, 'PRI_met,DER_mass_vis': 0.761852, 'PRI_met,DER_pt_h': 0.746764, 'PRI_met,DER_deltaeta_jet_jet': 0.7463919999999999, 'PRI_met,DER_mass_jet_jet': 0.7463919999999999, 'PRI_met,DER_prodeta_jet_jet': 0.7463919999999999, 'PRI_met,DER_deltar_tau_lep': 0.747788, 'PRI_met,DER_pt_tot': 0.746648, 'PRI_met,DER_sum_pt': 0.748304, 'PRI_met,DER_pt_ratio_lep_tau': 0.7491199999999999, 'PRI_met,DER_met_phi_centrality': 0.746564, 'PRI_met,DER_lep_eta_centrality': 0.7463919999999999, 'PRI_met,PRI_tau_pt': 0.751052, 'PRI_met,PRI_tau_eta': 0.7482599999999999, 'PRI_met,PRI_tau_phi': 0.746852, 'PRI_met,PRI_lep_pt': 0.74696, 'PRI_met,PRI_lep_eta': 0.7487119999999999, 'PRI_met,PRI_lep_phi': 0.7465079999999998, 'PRI_met,PRI_met_phi': 0.746312, 'PRI_met,PRI_met_sumet': 0.748228, 'PRI_met,PRI_jet_leading_pt': 0.7463919999999999, 'PRI_met,PRI_jet_leading_eta': 0.7463919999999999, 'PRI_met,PRI_jet_leading_phi': 0.7463919999999999, 'PRI_met,PRI_jet_subleading_pt': 0.7463919999999999, 'PRI_met,PRI_jet_subleading_eta': 0.7463919999999999, 'PRI_met,PRI_jet_subleading_phi': 0.7463919999999999, 'PRI_met,PRI_jet_all_pt': 0.7471880000000001, 'PRI_met_phi,DER_mass_MMC': 0.7468600000000001, 'PRI_met_phi,DER_mass_transverse_met_lep': 0.7462519999999999, 'PRI_met_phi,DER_mass_vis': 0.7589480000000001, 'PRI_met_phi,DER_pt_h': 0.7470959999999999, 'PRI_met_phi,DER_deltaeta_jet_jet': 0.7468600000000001, 'PRI_met_phi,DER_mass_jet_jet': 0.7468600000000001, 'PRI_met_phi,DER_prodeta_jet_jet': 0.7468600000000001, 'PRI_met_phi,DER_deltar_tau_lep': 0.748756, 'PRI_met_phi,DER_pt_tot': 0.7470480000000002, 'PRI_met_phi,DER_sum_pt': 0.748236, 'PRI_met_phi,DER_pt_ratio_lep_tau': 0.7494080000000001, 'PRI_met_phi,DER_met_phi_centrality': 0.74684, 'PRI_met_phi,DER_lep_eta_centrality': 0.7468600000000001, 'PRI_met_phi,PRI_tau_pt': 0.750772, 'PRI_met_phi,PRI_tau_eta': 0.748504, 'PRI_met_phi,PRI_tau_phi': 0.746944, 'PRI_met_phi,PRI_lep_pt': 0.7474480000000001, 'PRI_met_phi,PRI_lep_eta': 0.7487320000000001, 'PRI_met_phi,PRI_lep_phi': 0.7468680000000001, 'PRI_met_phi,PRI_met': 0.746312, 'PRI_met_phi,PRI_met_sumet': 0.7484039999999998, 'PRI_met_phi,PRI_jet_leading_pt': 0.7468600000000001, 'PRI_met_phi,PRI_jet_leading_eta': 0.7468600000000001, 'PRI_met_phi,PRI_jet_leading_phi': 0.7468600000000001, 'PRI_met_phi,PRI_jet_subleading_pt': 0.7468600000000001, 'PRI_met_phi,PRI_jet_subleading_eta': 0.7468600000000001, 'PRI_met_phi,PRI_jet_subleading_phi': 0.7468600000000001, 'PRI_met_phi,PRI_jet_all_pt': 0.7475160000000001, 'PRI_met_sumet,DER_mass_MMC': 0.7508079999999999, 'PRI_met_sumet,DER_mass_transverse_met_lep': 0.747668, 'PRI_met_sumet,DER_mass_vis': 0.75556, 'PRI_met_sumet,DER_pt_h': 0.7490479999999999, 'PRI_met_sumet,DER_deltaeta_jet_jet': 0.7508079999999999, 'PRI_met_sumet,DER_mass_jet_jet': 0.7508079999999999, 'PRI_met_sumet,DER_prodeta_jet_jet': 0.7508079999999999, 'PRI_met_sumet,DER_deltar_tau_lep': 0.7468440000000001, 'PRI_met_sumet,DER_pt_tot': 0.7507560000000001, 'PRI_met_sumet,DER_sum_pt': 0.746972, 'PRI_met_sumet,DER_pt_ratio_lep_tau': 0.7564200000000001, 'PRI_met_sumet,DER_met_phi_centrality': 0.750644, 'PRI_met_sumet,DER_lep_eta_centrality': 0.7508079999999999, 'PRI_met_sumet,PRI_tau_pt': 0.74766, 'PRI_met_sumet,PRI_tau_eta': 0.7469240000000001, 'PRI_met_sumet,PRI_tau_phi': 0.747736, 'PRI_met_sumet,PRI_lep_pt': 0.747116, 
    'PRI_met_sumet,PRI_lep_eta': 0.7470160000000001, 'PRI_met_sumet,PRI_lep_phi': 0.7479439999999999, 'PRI_met_sumet,PRI_met': 0.748228, 'PRI_met_sumet,PRI_met_phi': 0.7484039999999998, 'PRI_met_sumet,PRI_jet_leading_pt': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_leading_eta': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_leading_phi': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_subleading_pt': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_subleading_eta': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_subleading_phi': 0.7508079999999999, 'PRI_met_sumet,PRI_jet_all_pt': 0.748092, 'PRI_jet_leading_pt,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_leading_pt,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_leading_pt,DER_mass_vis': 0.777992, 'PRI_jet_leading_pt,DER_pt_h': 0.747372, 'PRI_jet_leading_pt,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_pt,DER_mass_jet_jet': 0.7468640000000001, 'PRI_jet_leading_pt,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_pt,DER_deltar_tau_lep': 0.75034, 'PRI_jet_leading_pt,DER_pt_tot': 0.747136, 'PRI_jet_leading_pt,DER_sum_pt': 0.753956, 'PRI_jet_leading_pt,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_leading_pt,DER_met_phi_centrality': 0.746804, 'PRI_jet_leading_pt,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_tau_pt': 0.7611320000000001, 'PRI_jet_leading_pt,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_leading_pt,PRI_tau_phi': 0.746988, 'PRI_jet_leading_pt,PRI_lep_pt': 0.747748, 'PRI_jet_leading_pt,PRI_lep_eta': 0.750612, 'PRI_jet_leading_pt,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_met': 0.7463919999999999, 'PRI_jet_leading_pt,PRI_met_phi': 0.7468600000000001, 'PRI_jet_leading_pt,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_leading_pt,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_jet_subleading_pt': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_jet_leading_pt,PRI_jet_all_pt': 0.7519, 'PRI_jet_leading_eta,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_leading_eta,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_leading_eta,DER_mass_vis': 0.777992, 'PRI_jet_leading_eta,DER_pt_h': 0.747372, 'PRI_jet_leading_eta,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_eta,DER_mass_jet_jet': 0.7468640000000001, 'PRI_jet_leading_eta,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_eta,DER_deltar_tau_lep': 0.75034, 'PRI_jet_leading_eta,DER_pt_tot': 0.747136, 'PRI_jet_leading_eta,DER_sum_pt': 0.753948, 'PRI_jet_leading_eta,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_leading_eta,DER_met_phi_centrality': 0.746808, 'PRI_jet_leading_eta,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_tau_pt': 0.7611320000000001, 'PRI_jet_leading_eta,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_leading_eta,PRI_tau_phi': 0.746988, 'PRI_jet_leading_eta,PRI_lep_pt': 0.747748, 'PRI_jet_leading_eta,PRI_lep_eta': 0.750612, 'PRI_jet_leading_eta,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_met': 0.7463919999999999, 'PRI_jet_leading_eta,PRI_met_phi': 0.7468600000000001, 'PRI_jet_leading_eta,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_leading_eta,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_jet_subleading_pt': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_jet_leading_eta,PRI_jet_all_pt': 0.7519, 'PRI_jet_leading_phi,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_leading_phi,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_leading_phi,DER_mass_vis': 0.777992, 'PRI_jet_leading_phi,DER_pt_h': 0.747372, 'PRI_jet_leading_phi,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_phi,DER_mass_jet_jet': 0.7468600000000001, 'PRI_jet_leading_phi,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_leading_phi,DER_deltar_tau_lep': 0.75034, 'PRI_jet_leading_phi,DER_pt_tot': 0.747136, 'PRI_jet_leading_phi,DER_sum_pt': 0.75396, 'PRI_jet_leading_phi,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_leading_phi,DER_met_phi_centrality': 0.746808, 'PRI_jet_leading_phi,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_tau_pt': 0.7611320000000001, 'PRI_jet_leading_phi,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_leading_phi,PRI_tau_phi': 0.746988, 'PRI_jet_leading_phi,PRI_lep_pt': 0.747748, 'PRI_jet_leading_phi,PRI_lep_eta': 0.750612, 'PRI_jet_leading_phi,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_met': 0.7463919999999999, 'PRI_jet_leading_phi,PRI_met_phi': 0.7468600000000001, 'PRI_jet_leading_phi,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_leading_phi,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_jet_subleading_pt': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_jet_leading_phi,PRI_jet_all_pt': 0.7519, 'PRI_jet_subleading_pt,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_subleading_pt,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_subleading_pt,DER_mass_vis': 0.777992, 'PRI_jet_subleading_pt,DER_pt_h': 0.747372, 'PRI_jet_subleading_pt,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_pt,DER_mass_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_pt,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_pt,DER_deltar_tau_lep': 0.75034, 'PRI_jet_subleading_pt,DER_pt_tot': 0.747136, 'PRI_jet_subleading_pt,DER_sum_pt': 0.75396, 'PRI_jet_subleading_pt,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_subleading_pt,DER_met_phi_centrality': 0.746808, 'PRI_jet_subleading_pt,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_tau_pt': 0.7611320000000001, 'PRI_jet_subleading_pt,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_subleading_pt,PRI_tau_phi': 0.746988, 'PRI_jet_subleading_pt,PRI_lep_pt': 0.747748, 'PRI_jet_subleading_pt,PRI_lep_eta': 0.750612, 'PRI_jet_subleading_pt,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_met': 0.7463919999999999, 'PRI_jet_subleading_pt,PRI_met_phi': 0.7468600000000001, 'PRI_jet_subleading_pt,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_subleading_pt,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_jet_subleading_pt,PRI_jet_all_pt': 0.7519, 'PRI_jet_subleading_eta,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_subleading_eta,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_subleading_eta,DER_mass_vis': 0.777992, 'PRI_jet_subleading_eta,DER_pt_h': 0.747372, 'PRI_jet_subleading_eta,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_eta,DER_mass_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_eta,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_eta,DER_deltar_tau_lep': 0.7503560000000001, 'PRI_jet_subleading_eta,DER_pt_tot': 0.747136, 'PRI_jet_subleading_eta,DER_sum_pt': 0.7539960000000001, 'PRI_jet_subleading_eta,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_subleading_eta,DER_met_phi_centrality': 0.746808, 'PRI_jet_subleading_eta,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_tau_pt': 0.7611360000000001, 'PRI_jet_subleading_eta,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_subleading_eta,PRI_tau_phi': 0.746988, 'PRI_jet_subleading_eta,PRI_lep_pt': 0.747748, 'PRI_jet_subleading_eta,PRI_lep_eta': 0.750612, 'PRI_jet_subleading_eta,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_met': 0.7463919999999999, 'PRI_jet_subleading_eta,PRI_met_phi': 0.7468600000000001, 'PRI_jet_subleading_eta,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_subleading_eta,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_jet_subleading_pt': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_jet_subleading_phi': 0.7468640000000001, 'PRI_jet_subleading_eta,PRI_jet_all_pt': 0.7519, 'PRI_jet_subleading_phi,DER_mass_MMC': 0.7468640000000001, 'PRI_jet_subleading_phi,DER_mass_transverse_met_lep': 0.745404, 'PRI_jet_subleading_phi,DER_mass_vis': 0.777992, 'PRI_jet_subleading_phi,DER_pt_h': 0.747372, 'PRI_jet_subleading_phi,DER_deltaeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_phi,DER_mass_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_phi,DER_prodeta_jet_jet': 0.7468640000000001, 'PRI_jet_subleading_phi,DER_deltar_tau_lep': 0.75034, 'PRI_jet_subleading_phi,DER_pt_tot': 0.747136, 'PRI_jet_subleading_phi,DER_sum_pt': 0.753956, 'PRI_jet_subleading_phi,DER_pt_ratio_lep_tau': 0.7527440000000001, 'PRI_jet_subleading_phi,DER_met_phi_centrality': 0.746808, 'PRI_jet_subleading_phi,DER_lep_eta_centrality': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_tau_pt': 0.7611320000000001, 'PRI_jet_subleading_phi,PRI_tau_eta': 0.7493679999999999, 'PRI_jet_subleading_phi,PRI_tau_phi': 0.746988, 'PRI_jet_subleading_phi,PRI_lep_pt': 0.747748, 'PRI_jet_subleading_phi,PRI_lep_eta': 0.750612, 'PRI_jet_subleading_phi,PRI_lep_phi': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_met': 0.7463919999999999, 'PRI_jet_subleading_phi,PRI_met_phi': 0.7468600000000001, 'PRI_jet_subleading_phi,PRI_met_sumet': 0.7508079999999999, 'PRI_jet_subleading_phi,PRI_jet_leading_pt': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_jet_leading_eta': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_jet_leading_phi': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_jet_subleading_pt': 0.7468640000000001, 
    'PRI_jet_subleading_phi,PRI_jet_subleading_eta': 0.7468640000000001, 'PRI_jet_subleading_phi,PRI_jet_all_pt': 0.7519, 'PRI_jet_all_pt,DER_mass_MMC': 0.7519, 'PRI_jet_all_pt,DER_mass_transverse_met_lep': 0.7457840000000001, 'PRI_jet_all_pt,DER_mass_vis': 0.768128, 'PRI_jet_all_pt,DER_pt_h': 0.7471759999999998, 'PRI_jet_all_pt,DER_deltaeta_jet_jet': 0.7519, 'PRI_jet_all_pt,DER_mass_jet_jet': 0.751896, 'PRI_jet_all_pt,DER_prodeta_jet_jet': 0.7519, 'PRI_jet_all_pt,DER_deltar_tau_lep': 0.747824, 'PRI_jet_all_pt,DER_pt_tot': 0.749128, 'PRI_jet_all_pt,DER_sum_pt': 0.7493879999999999, 'PRI_jet_all_pt,DER_pt_ratio_lep_tau': 0.756, 'PRI_jet_all_pt,DER_met_phi_centrality': 0.7489, 'PRI_jet_all_pt,DER_lep_eta_centrality': 0.7519, 'PRI_jet_all_pt,PRI_tau_pt': 0.753144, 'PRI_jet_all_pt,PRI_tau_eta': 0.74758, 'PRI_jet_all_pt,PRI_tau_phi': 0.747044, 'PRI_jet_all_pt,PRI_lep_pt': 0.746904, 'PRI_jet_all_pt,PRI_lep_eta': 0.748336, 'PRI_jet_all_pt,PRI_lep_phi': 0.747084, 'PRI_jet_all_pt,PRI_met': 0.7471880000000001, 'PRI_jet_all_pt,PRI_met_phi': 0.7475160000000001, 'PRI_jet_all_pt,PRI_met_sumet': 0.748092, 'PRI_jet_all_pt,PRI_jet_leading_pt': 0.7519, 'PRI_jet_all_pt,PRI_jet_leading_eta': 0.7519, 'PRI_jet_all_pt,PRI_jet_leading_phi': 0.7519, 'PRI_jet_all_pt,PRI_jet_subleading_pt': 0.7519, 'PRI_jet_all_pt,PRI_jet_subleading_eta': 0.7519, 'PRI_jet_all_pt,PRI_jet_subleading_phi': 0.7519}
arr = [(key, val) for key, val in d.items()]
combs = list(map(lambda x: x[0], sorted(list(filter(lambda x: 'mass_vis' not in x[0], arr)), key=lambda x: -x[1])))
combs

In [ ]:
def train_model(y, x):
  w, _ = least_squares(y, x)
  #w, loss = least_squares(y, x)
  def predict(features):
    result = features @ w
    result[np.where(result < 0)] = -1
    result[np.where(result >= 0)] = 1
    return result
  return predict
_ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)

In [ ]:
func_arr = [['DER_mass_transverse_met_lep', 0.745404], ['DER_mass_vis', 0.777992], ['DER_pt_h', 0.747372], ['DER_deltaeta_jet_jet', 0.722948], ['DER_mass_jet_jet', 0.7468640000000001], ['DER_prodeta_jet_jet', 0.7428480000000001], ['DER_deltar_tau_lep', 0.7503439999999999], ['DER_pt_tot', 0.747136], ['DER_sum_pt', 0.7539440000000001], ['DER_pt_ratio_lep_tau', 0.7527440000000001], ['DER_met_phi_centrality', 0.746808], ['DER_lep_eta_centrality', 0.746872], ['PRI_tau_pt', 0.7611320000000001], ['PRI_tau_eta', 0.7493679999999999], ['PRI_tau_phi', 0.746988], ['PRI_lep_pt', 0.747748], ['PRI_lep_eta', 0.750612], ['PRI_lep_phi', 0.7468640000000001], ['PRI_met', 0.7463919999999999], ['PRI_met_phi', 0.7468600000000001], ['PRI_met_sumet', 0.7508079999999999], ['PRI_jet_leading_pt', 0.7468640000000001], ['PRI_jet_leading_eta', 0.7468640000000001], ['PRI_jet_leading_phi', 0.7468640000000001], ['PRI_jet_subleading_pt', 0.7468640000000001], ['PRI_jet_subleading_eta', 0.7468640000000001], ['PRI_jet_subleading_phi', 0.7468640000000001], ['PRI_jet_all_pt', 0.7519]]
sorted(func_arr, key=lambda x: -x[1])

In [ ]:
print(np.max(tx, axis=0))
print(np.std(tx, axis=0))

In [ ]:
for k in range(1, 6):
  gamma = 10 ** (-k)
  def train_model(y, x):
    w, _ = least_squares(y, x)
    #w, loss = least_squares(y, x)
    def predict(features):
      result = features @ w
      result[np.where(result < 0)] = -1
      result[np.where(result >= 0)] = 1
      return result
    return predict
  _ = validation.cross_validation(ty, tx, train_model, 10, verbose=True)

In [ ]:

ks = []
accs = []
for k in range(50, 100):
  lambda_ = 10 ** (-(k / 5))
  print('Lambda:', lambda_)  
  def train_model(y, x):
    #w, _ = least_squares_GD(y, x, np.zeros((x.shape[1],)), 100, gamma=gamma)
    #w, loss = least_squares(y, x)
    w, _ = ridge_regression(y, x, lambda_)
    def predict(features):
      result = features @ w
      result[np.where(result < 0)] = -1
      result[np.where(result >= 0)] = 1
      return result
    return predict
  acc, fbeta = validation.cross_validation(ty, tx, train_model, 10, verbose=True)
  ks.append(k)
  accs.append(np.mean(acc))
  print()


In [ ]:
import matplotlib.pyplot as plt

print(np.argmax(accs))
print(np.max(accs))
print(ks[np.argmax(accs)])

plt.plot(ks, accs)
plt.xlabel(r'$-5\log_{10} (\lambda)$')
plt.show()


In [ ]:
ridge_regression_lambda = 10 ** (-(77 / 5))
y, x, ids, cols = data_io.load_csv_data(f'{DATA_FILE_PREFIX}train.csv')

import copy

tx, ty = prepare_data(x, y)

w, loss = least_squares(ty, tx)
def predict(features):
      result = features @ w
      result[np.where(result < 0)] = -1
      result[np.where(result >= 0)] = 1
      return result

loss


In [ ]:
y, x, ids, _ = data_io.load_csv_data(f'{DATA_FILE_PREFIX}test.csv')

tx, ty = prepare_data(x, y)

labels = predict(tx)

In [ ]:
data_io.create_csv_submission(ids, labels, '812accsubmission.csv')